In [ ]:
# !pip install langchain langchain-community langchain-text-splitters langchain-huggingface langchain-chroma pypdf transformers torch accerlate timm

  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_huggingface-0.3.0-py3-none-any.whl.metadata (996 bytes)
  Using cached langchain_chroma-0.2.4-py3-none-any.whl.metadata (1.1 kB)
  Using cached pypdf-5.7.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached torch-2.7.1-cp310-cp310-win_amd64.whl.metadata (28 kB)
  Using cached langchain_core-0.3.67-py3-none-any.whl.metadata (5.8 kB)
  Using cached langsmith-0.4.4-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.41-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.embeddings import Embeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModel
import torch
from typing import List

from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_chroma import Chroma

from langchain.chains import create_retrieval_chain

c:\Users\user\miniconda3\envs\llm_p_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPU ÌôïÏù∏

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"ÏÇ¨Ïö©Ìï† Ïû•Ïπò: {device}")
if device == "cuda":
    print(f"ÏÇ¨Ïö© Í∞ÄÎä•Ìïú GPU Í∞úÏàò: {torch.cuda.device_count()}")
    print(f"ÌòÑÏû¨ GPU Ïû•Ïπò: {torch.cuda.get_device_name(0)}")
print("-" * 50)

ÏÇ¨Ïö©Ìï† Ïû•Ïπò: cuda
ÏÇ¨Ïö© Í∞ÄÎä•Ìïú GPU Í∞úÏàò: 1
ÌòÑÏû¨ GPU Ïû•Ïπò: NVIDIA GeForce RTX 4090
--------------------------------------------------


## 1. Îç∞Ïù¥ÌÑ∞ Ï§ÄÎπÑ (Î°úÎçî, Ïä§ÌîåÎ¶øÌÑ∞)

In [ ]:
print("[1Îã®Í≥Ñ] PDF Îç∞Ïù¥ÌÑ∞ Î°úÎî© Î∞è Î∂ÑÌï† ÏãúÏûë")

pdf_path = r'C:\Users\user\Documents\workspace_llm\workspace_llm_p_2\Í∏∞Í¥ÄÏÇ¨3Í∏â_ocr.pdf'

if not os.path.exists(pdf_path):
    print(f"Ïò§Î•ò: PDF ÌååÏùºÏùÑ Ï∞æÏùÑ Ïàò ÏóÜÏäµÎãàÎã§. Í≤ΩÎ°úÎ•º ÌôïÏù∏ÌïòÏÑ∏Ïöî: {pdf_path}")
    exit()

loader = PyPDFLoader(pdf_path)
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)
print(f"PDF Î°úÎî© Î∞è Î∂ÑÌï† ÏôÑÎ£å. Ï¥ù Î∂ÑÌï†Îêú Î¨∏ÏÑú Ï°∞Í∞Å Ïàò: {len(all_splits)}")
print("-" * 50)

[1Îã®Í≥Ñ] PDF Îç∞Ïù¥ÌÑ∞ Î°úÎî© Î∞è Î∂ÑÌï† ÏãúÏûë
PDF Î°úÎî© Î∞è Î∂ÑÌï† ÏôÑÎ£å. Ï¥ù Î∂ÑÌï†Îêú Î¨∏ÏÑú Ï°∞Í∞Å Ïàò: 1038
--------------------------------------------------


## 2. ÏûÑÎ≤†Îî© Î™®Îç∏ (Ïª§Ïä§ÌÖÄ ÌÅ¥ÎûòÏä§)

In [ ]:
model_name = "BAAI/bge-m3"

embedding_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True} # Î≤°ÌÑ∞ Ï†ïÍ∑úÌôî
)
print("Î≤°ÌÑ∞Ï†ïÍ∑úÌôî (ÏûÑÎ≤†Îî©) Ï§ÄÎπÑ ÏôÑÎ£å")

Î≤°ÌÑ∞Ï†ïÍ∑úÌôî (ÏûÑÎ≤†Îî©) Ï§ÄÎπÑ ÏôÑÎ£å


## 3. Î≤°ÌÑ∞ dbÏóê Ï†ÄÏû• (ÏûÑÎ≤†Îî©, Î¶¨Ìä∏Î¶¨Î≤Ñ ÏÑ§Ï†ï)

In [ ]:
print("[3Îã®Í≥Ñ] Î≤°ÌÑ∞ DB ÏÉùÏÑ± Î∞è Retriever ÏÑ§Ï†ï ÏãúÏûë")


persist_directory = './chroma_db_gemma3_n_test_v1'

if not os.path.exists(persist_directory):
    print("ÏÉàÎ°úÏö¥ Chroma Î≤°ÌÑ∞ Ïä§ÌÜ†Ïñ¥Î•º ÏÉùÏÑ±Ìï©ÎãàÎã§. (ÏãúÍ∞ÑÏù¥ ÏÜåÏöîÎê† Ïàò ÏûàÏäµÎãàÎã§...)")
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding_function,
        persist_directory=persist_directory
    )
    print("‚úÖ Î≤°ÌÑ∞ Ïä§ÌÜ†Ïñ¥ ÏÉùÏÑ± Î∞è Ï†ÄÏû• ÏôÑÎ£å.")
else:
    print("Í∏∞Ï°¥ Chroma Î≤°ÌÑ∞ Ïä§ÌÜ†Ïñ¥Î•º Î°úÎìúÌï©ÎãàÎã§.")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_function
    )
    print("‚úÖ Î≤°ÌÑ∞ Ïä§ÌÜ†Ïñ¥ Î°úÎìú ÏôÑÎ£å.")

retriever = vectorstore.as_retriever(search_kwargs={"k": 7})
print("‚úÖ Retriever ÏÑ§Ï†ï ÏôÑÎ£å. (ÏÉÅÏúÑ 7Í∞ú Î¨∏ÏÑú Í≤ÄÏÉâ)")
print("-" * 50)

[3Îã®Í≥Ñ] Î≤°ÌÑ∞ DB ÏÉùÏÑ± Î∞è Retriever ÏÑ§Ï†ï ÏãúÏûë
Í∏∞Ï°¥ Chroma Î≤°ÌÑ∞ Ïä§ÌÜ†Ïñ¥Î•º Î°úÎìúÌï©ÎãàÎã§.
‚úÖ Î≤°ÌÑ∞ Ïä§ÌÜ†Ïñ¥ Î°úÎìú ÏôÑÎ£å.
‚úÖ Retriever ÏÑ§Ï†ï ÏôÑÎ£å. (ÏÉÅÏúÑ 7Í∞ú Î¨∏ÏÑú Í≤ÄÏÉâ)
--------------------------------------------------


## llmÎ™®Îç∏ ÏßÄÏ†ï

In [ ]:
print("[4Îã®Í≥Ñ] LLM Ï§ÄÎπÑ ÏãúÏûë")

llm_model_id = "google/gemma-3n-E2B-it"

tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager" # triton ÏÑ§Ï†ï : Í∞ÄÏû• Í∏∞Î≥∏Ï†ÅÏù∏ pytorch Ïñ¥ÌÖêÏÖò Íµ¨ÌòÑ
)

pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.2, 
    return_full_text=False 
)

llm = HuggingFacePipeline(pipeline=pipe)
print("‚úÖ LLM Ï§ÄÎπÑ ÏôÑÎ£å : {llm_model_id}")
print("-" * 50)

[4Îã®Í≥Ñ] LLM Ï§ÄÎπÑ ÏãúÏûë


Loading checkpoint shards: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 3/3 [00:03<00:00,  1.10s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


‚úÖ LLM Ï§ÄÎπÑ ÏôÑÎ£å : {llm_model_id}
--------------------------------------------------


## ÌîÑÎ°¨ÌîÑÌä∏ Î∞è RAG Chain ÏÉùÏÑ±


In [ ]:
print("[5Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ ÏÉùÏÑ± ÏãúÏûë")

prompt = ChatPromptTemplate.from_template("""<start_of_turn>user
You are a Q&A bot that must answer the user's ‚ÄúQuestion‚Äù using only the provided ‚ÄúRetrieved Documents.‚Äù

**Your mission:**
1. Fully comprehend and analyze the content of the ‚ÄúRetrieved Documents.‚Äù
2. Never use your prior knowledge or any external information. You must base your answers solely on what is explicitly stated in the ‚ÄúRetrieved Documents.‚Äù
3. If the answer to the question clearly exists within the ‚ÄúRetrieved Documents,‚Äù locate it and provide the correct answer along with a detailed explanation.
4. If you cannot find the answer to the question within the ‚ÄúRetrieved Documents,‚Äù do not guess‚Äîhonestly respond, ‚ÄúI cannot answer based on the provided documents.‚Äù


**think step-by-step inside a <thinking> block to analyze the problem:**
1.  Identify the core question the user is asking.
2.  Scan the 'Context' to find the multiple-choice question and its options that match the user's query.
3.  Locate any text in the 'Context' that serves as an explanation, a definition, or a clue to the correct answer.
4.  Determine the correct option based on the evidence found.
5.  Formulate a clear explanation based on your findings.

After your thinking process, generate the final user-facing answer in Korean, strictly following the format below.

## Final Answer Format:
**Final Answer :**
[Ï†ïÎãµ Î≥¥Í∏∞ÏôÄ ÎÇ¥Ïö©ÏùÑ Î™ÖÏãúÌï©ÎãàÎã§. Ïòà: ÏÇ¨. Í≥†ÏÜçÏÑ†ÏóêÏÑúÎäî Í≤ΩÏÇ¨Í∞Ä ÏûëÏùÄ Ï∂îÏßÑÍ∏∞Î•º ÏÇ¨Ïö©ÌïúÎã§.]

**Ìï¥ÏÑ§:**
[Ï∞æÏïÑÎÇ∏ Í∑ºÍ±∞Î•º Î∞îÌÉïÏúºÎ°ú Ï†ïÎãµÏù∏ Ïù¥Ïú†ÏôÄ Ïò§ÎãµÏù∏ Ïù¥Ïú†Î•º ÏÉÅÏÑ∏ÌïòÍ≤å ÏÑ§Î™ÖÌï©ÎãàÎã§. Ïòà: ÌîÑÎ°úÌé†Îü¨ ÎÇ†Í∞úÎ•º ÌõÑÎ∞©Í≤ΩÏÇ¨ ÌïòÎ©¥ Ï∂îÏßÑÍ∏∞Ïùò Ìö®Ïú®ÏùÄ Ï¶ùÍ∞Ä ÌïúÎã§.]

---
**IMPORTANT**: If the provided 'Context' is insufficient to determine the correct answer and provide a clear explanation, your final answer must be: "Ï†úÍ≥µÎêú Ï†ïÎ≥¥ÎßåÏúºÎ°úÎäî ÏßàÎ¨∏Ïóê ÎåÄÌïú Ï†ïÌôïÌïú Ìï¥ÏÑ§Ïù¥ÎÇò Ï†ïÎãµÏùÑ Ï∞æÏùÑ Ïàò ÏóÜÏäµÎãàÎã§." Do not use any external knowledge.

# Context:
{context}

# Question:
{input}<end_of_turn>
<start_of_turn>model
<thinking>
1. Identify the core question.
2. Locate the matching question and options in the Context.
3. Find evidence (definitions, explanations, clues) in the Context.
4. Select the correct option based on evidence.
5. Prepare a clear explanation.
</thinking>
""")

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)
print("‚úÖ RAG Ï≤¥Ïù∏ ÏÉùÏÑ± ÏôÑÎ£å.")
print("-" * 50)

[5Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ ÏÉùÏÑ± ÏãúÏûë
‚úÖ RAG Ï≤¥Ïù∏ ÏÉùÏÑ± ÏôÑÎ£å.
--------------------------------------------------


## Ïã§Ìñâ (invoke)

In [ ]:
print("[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ")
question = "Ìï¥ÏñëÌôòÍ≤ΩÍ¥ÄÎ¶¨Î≤ïÏóê ÏùòÌïú Í∏∞Î¶ÑÏò§ÏóºÎ∞©ÏßÄÏÑ§ÎπÑ Ï§ë Í∏∞Î¶ÑÎ∞õÏù¥Ïùò Í∏∞Ï§ÄÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ? Í∞Ä. Ïû¨ÏßàÏùÄ Í∏àÏÜçÏ†úÏùº Í≤É ÎÇò. ÏÑ†Î∞ïÏù¥ ÏÑ∏Î°úÎ∞©Ìñ•ÏúºÎ°ú 10ÎèÑ ÎòêÎäî Í∞ÄÎ°úÎ∞©Ìñ•ÏúºÎ°ú 22.5ÎèÑ Í≤ΩÏÇ¨ÌïòÎäî Í≤ΩÏö∞ÏóêÎèÑ Í∏∞Î¶ÑÏù¥ ÎÑòÏπòÍ±∞ÎÇò ÌùêÎ•¥ÏßÄ ÏïäÎäî Íµ¨Ï°∞Ïùº Í≤É ÏÇ¨. Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ÏÑ§Ïπò ÏïÑ. Î™®Îì† ÌÉ±ÌÅ¨Ïùò ÏóêÏñ¥Î≤§Ìä∏(tank air vent)ÏóêÎäî Î∞òÎìúÏãú Í∏∞Î¶Ñ Î∞õÏù¥Í∞Ä ÏûàÏùÑ Í≤É"

print(f"üí¨ ÏßàÎ¨∏: {question}\n")

response = rag_chain.invoke({"input": question})

print("--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---")
for i, doc in enumerate(response["context"]):
    print(f"Î¨∏ÏÑú {i+1}: {doc.page_content[:150]}...")
    
print("-" * 30)
print("--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---")
print(response["answer"])

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: Ìï¥ÏñëÌôòÍ≤ΩÍ¥ÄÎ¶¨Î≤ïÏóê ÏùòÌïú Í∏∞Î¶ÑÏò§ÏóºÎ∞©ÏßÄÏÑ§ÎπÑ Ï§ë Í∏∞Î¶ÑÎ∞õÏù¥Ïùò Í∏∞Ï§ÄÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ? Í∞Ä. Ïû¨ÏßàÏùÄ Í∏àÏÜçÏ†úÏùº Í≤É ÎÇò. ÏÑ†Î∞ïÏù¥ ÏÑ∏Î°úÎ∞©Ìñ•ÏúºÎ°ú 10ÎèÑ ÎòêÎäî Í∞ÄÎ°úÎ∞©Ìñ•ÏúºÎ°ú 22.5ÎèÑ Í≤ΩÏÇ¨ÌïòÎäî Í≤ΩÏö∞ÏóêÎèÑ Í∏∞Î¶ÑÏù¥ ÎÑòÏπòÍ±∞ÎÇò ÌùêÎ•¥ÏßÄ ÏïäÎäî Íµ¨Ï°∞Ïùº Í≤É ÏÇ¨. Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ÏÑ§Ïπò ÏïÑ. Î™®Îì† ÌÉ±ÌÅ¨Ïùò ÏóêÏñ¥Î≤§Ìä∏(tank air vent)ÏóêÎäî Î∞òÎìúÏãú Í∏∞Î¶Ñ Î∞õÏù¥Í∞Ä ÏûàÏùÑ Í≤É

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: ÎπÑ Ï§ë Í∏∞Î¶ÑÎ∞õÏù¥Ïùò Í∏∞Ï§ÄÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ? 
Í∞Ä. Ïû¨ÏßàÏùÄ Í∏àÏÜçÏ†úÏùº Í≤É
ÎÇò. ÏÑ†Î∞ïÏù¥ ÏÑ∏Î°úÎ∞©Ìñ•ÏúºÎ°ú 10ÎèÑ ÎòêÎäî Í∞ÄÎ°úÎ∞©Ìñ• 
ÏúºÎ°ú 22.5ÎèÑ Í≤ΩÏÇ¨ÌïòÎäî Í≤ΩÏö∞ÏóêÎèÑ Í∏∞Î¶ÑÏù¥ 
ÎÑòÏπòÍ±∞ÎÇò ÌùêÎ•¥ÏßÄ ÏïäÎäî Íµ¨Ï°∞Ïùº Í≤É
ÏÇ¨. Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ 
ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ...
Î¨∏ÏÑú 2: ÏßÅÎ¨¥ÏùºÎ∞ò Ï≤ôÏ§ë ÏòàÏÉÅ Î¨úÏ°î
Í∞Ä ÏàòÎ©¥ÏúÑÎ°ú Ïò¨ÎùºÏò¨ Í∞ÄÎä•ÏÑ±Ïù¥ Ïûà

1Ïû•-1Î≤à ÌïúÍµ≠Ïñ¥ Î¨∏Ï†ú, Ï†ïÎãµ(26Ï¥à), Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ O

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: ÎîîÏ†§Í∏∞Í¥ÄÏóêÏÑú Ïó∞Î£åÏÜåÎπÑÏú®Ïù¥ÎûÄ? Í∞Ä. Í∏∞Í¥ÄÏù¥ 1ÏãúÍ∞ÑÏóê ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâ ÎÇò. Ïó∞Î£åÏùò ÏãúÍ∞ÑÎãπ Î∞úÏó¥Îüâ ÏÇ¨. Í∏∞Í¥ÄÏù¥ 1ÏãúÍ∞ÑÎãπ 1ÎßàÎ†•ÏùÑ ÏñªÍ∏∞ ÏúÑÌï¥ ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâ ÏïÑ. Í∏∞Í¥ÄÏù¥ 1Ïã§Îü∞ÎçîÎãπ 1ÏãúÍ∞ÑÏóê ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâ

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: PART
01Ï†ÅÏ§ë ÏòàÏÉÅ Î¨∏Ï†ú
001 ‚ñ∫ ÎîîÏ†§Í∏∞Í¥ÄÏóêÏÑú Ïó∞Î£åÏÜåÎπÑÏú®Ïù¥ÎûÄ?
Í∞Ä. Í∏∞Í¥ÄÏù¥ 1ÏãúÍ∞ÑÏóê ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâ
ÎÇò. Ïó∞Î£åÏùò ÏãúÍ∞ÑÎãπ Î∞úÏó¥Îüâ
ÏÇ¨. Í∏∞Í¥ÄÏù¥ 1ÏãúÍ∞ÑÎãπ 1ÎßàÎ†•ÏùÑ ÏñªÍ∏∞ ÏúÑÌï¥ ÏÜåÎπÑÌïò 
Îäî Ïó∞Î£åÎüâ
ÏïÑ. Í∏∞Í¥ÄÏù¥ 1Ïã§Î¶∞ÎçîÎãπ 1ÏãúÍ∞ÑÏóê ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâ
/ Ïó∞Î£åÏÜåÎπÑÏú®: Ïó∞Î£åÏÜåÎπÑÏú®ÏùÄ...
Î¨∏ÏÑú 2: Î∞∞Í∏∞ÌÑ∞Îπà Ìö®Ïú®Ïù¥ ÎÜíÎã§.
019 ‚ñ∫ ÎîîÏ†§Í∏∞Í¥ÄÏùò Í≥†Ï∂úÎ†•ÌôîÎ•º ÏúÑÌïú Î∞©Î≤ï Ï§ë 
ÎπÑÏ∂úÎ†•ÏùÑ Ï¶ùÎåÄÏãúÌÇ§Îäî Î∞©Î≤ïÏù¥ ÏïÑÎãå Í≤ÉÏùÄ?
Í∞Ä. ÌñâÏ†ïÏö©Ï†ÅÏùÑ ÌÅ¨Í≤å ÌïúÎã§.
ÎÇò. ÌöåÏ†ÑÏàòÎ•º ÎÜíÏù∏Îã§.
ÏÇ¨. ÌèâÍ∑†Ïú†Ìö®ÏïïÎ†•ÏùÑ ÎÜíÏù∏Îã§.
ÏïÑ. Í≥ºÍ∏âÍ∏∞Ïùò Ìö®Ïú®ÏùÑ ÎÜíÏù∏Îã§.
/ ÎπÑÏ∂úÎ†•: ÎèôÎ†• Í∏∞Í≥ÑÏñ¥ÏÑú Ïó∞Î£åÍ∞Ä Îã®ÏúÑÎüâ Îãπ ÏÉù...
Î¨∏ÏÑú 3: Í∏∞Í¥Ä I Ï†ÅÏ§ë ÏòàÏÉÅ Î¨úÏ°î
Í∞Ä. Ï†êÎèÑÏßÄÏàò ÎÇò. ÎπÑÏ†êÎèÑ
ÏÇ¨. Ï†àÎåÄÏ†êÎèÑ ÏïÑ. ÎèôÏ†êÎèÑ
/ ÎèôÏ†êÎèÑ : Ï†êÏÑ±ÎèÑÎ•º Î∞ÄÎèÑÎ°ú ÎÇòÎàà Í∞í
130 ‚ñ∫ Îã§Ïùå Ï§ë Ï†ÄÏÜç ÎîîÏ†§Í∏∞Í¥ÄÏóêÏÑú Í∞ÄÏû• ÎßéÏù¥ 
ÏÇ¨Ïö©ÎêòÎäî Ïó∞ÏÜåÏã§ ÌòïÌÉúÎäî?
Í∞Ä. ÏßÅÏ†ëÎ∂ÑÏÇ¨Ïãù Ïó∞ÏÜåÏã§
ÎÇò. ÏòàÏó∞ÏÜåÏã§Ïãù Ïó∞ÏÜåÏã§ 
ÏÇ¨. ÏôÄÏã§Ïãù Ïó∞ÏÜåÏã§ 
ÏïÑ. ...
Î¨∏ÏÑú 4: 04. ÎÇ¥Ïó∞Í∏∞Í¥ÄÏùò Ïó¥Ìö®Ïú®Ïóê ÎåÄÌïú ÏÑ§Î™ÖÏúºÎ°ú Ïò≥ÏùÄ Í≤É 
ÏùÄ?
Í∞Ä. ÏµúÍ≥†ÏïïÎ†•Ïù¥ ÏùºÏ†ïÌï† ÎïåÎäî Ïò§ÌÜ† ÏÇ¨Ïù¥ÌÅ¥Ïùò 
Ìö®Ïú®Ïù¥ Í∞ÄÏû• Ï¢ãÎã§.
ÎÇò. ÏµúÍ≥†ÏïïÎ†•Ïù¥ ÏùºÏ†ïÌï† ÎïåÎäî ÎîîÏ†§ ÏÇ¨Ïù¥ÌÅ¥Ïùò 
Ìö®Ïú®Ïù¥ Í∞ÄÏû• Ï¢ãÎã§.
ÏÇ¨. ÏµúÍ≥†ÏïïÎ†•Ïù¥ ÏùºÏ†ïÌï† ÎïåÎäî ÏÇ¨Î∞îÎç∞ ÏÇ¨Ïù¥ÌÅ¥Ïùò 
Ìö®Ïú®Ïù¥ Í∞ÄÏû• Ï¢ãÎã§.
ÏïÑ. ÏµúÍ≥†ÏïïÎ†•Ïù¥ ...
Î¨∏ÏÑú 5: Í∞Ä‚ñ†. A : mechanical efficiency,
B : specific fuel oil consumption 
ÎÇò. A : thermal efficiency,
B : specific fuel oil consumption 
Ïãú‚Äô. A : mechanical eff...
Î¨∏ÏÑú 6: Í∏∞Í¥Ä I Ï†ÅÏ§ë ÏòàÏÉÅ Î¨∏Ï†ú
Í∞Ä. ÏÑ∏ÌÉÑÍ∞ÄÍ∞Ä ÎÇÆÏùÄ Í∏∞Î¶ÑÏùÑ ÏÇ¨Ïö©ÌïúÎã§.
ÎÇò. ÏïïÏ∂ïÎπÑÎ•º Ï¶ùÍ∞ÄÏãúÌÇ®Îã§.
ÏÇ¨. Ï∞©ÌôîÏãúÍπåÏßÄ Ïó∞Î£å Î∂ÑÏÇ¨ÎüâÏùÑ Ï†ÅÍ≤å ÌïúÎã§. 
ÏïÑ. Ïã§Î¶∞ÎçîÏùò ÎÉâÍ∞ÅÏàò Ïò®ÎèÑÎ•º ÎÜíÏù∏Îã§.
/ ÏÑ∏ÌÉÑÍ∞Ä ÎÜíÏùÄ Í∏∞Î¶ÑÏùÑ ÏÇ¨Ïö©ÌïòÏó¨Ïïº ÌïúÎã§.
062 ‚ñ∫ ÎîîÏ†§Í∏∞Í¥ÄÏóêÏÑú Î∞∏Î∏å ÌÉúÌïè Í∞ÑÍ≤©ÏùÑ Ï£ºÎäî Ïù¥Ïú† 
Îäî...
Î¨∏ÏÑú 7: Ï†ú 2 Ìöå [ Í∏∞Í¥Ä I ]
01. ÎîîÏ†§Í∏∞Í¥ÄÏóêÏÑú Ï≤¥Ï†ÅÌö®Ïú®Ïùò Ï†ÄÌïò ÏõêÏù∏ÏúºÎ°ú Ïò≥ÏßÄ 
ÏïäÏùÄÍ≤ÉÏùÄ?
Í∞Ä. Ìù°ÏûÖÍ≥µÍ∏∞Ïùò Ïò®ÎèÑÍ∞Ä ÎÜíÏùÑ Îïå 
ÎÇò. Ïã§Î¶∞Îçî ÎÇ¥Ïùò Ïò®ÎèÑÍ∞Ä ÎÜíÏùÑ Îïå 
ÏÇ¨. Ìù°‚ñ†Î∞∞Í∏∞Î∞∏Î∏åÏùò Í∞úÌèêÏãúÍ∏∞Í∞Ä Ï†ÅÎãπÌïòÏßÄ Ïïä 
ÏùÑ Îïå
ÏïÑ. Ïã§Î¶∞ÎçîÏùò Î∞∞ÏïïÏù¥ ÎÇÆÏùÑ Îïå
02. ÎÇ¥Ïó∞Í∏∞Í¥ÄÏùò Ïù¥Î°† Í≥µÍ∏∞...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer Format:
**Final Answer :** ÏÇ¨. Í∏∞Í¥ÄÏù¥ 1ÏãúÍ∞ÑÎãπ 1ÎßàÎ†•ÏùÑ ÏñªÍ∏∞ ÏúÑÌï¥ ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâ

**Ìï¥ÏÑ§:**
ÏßàÎ¨∏ÏùÄ ÎîîÏ†§Í∏∞Í¥ÄÏóêÏÑú Ïó∞Î£åÏÜåÎπÑÏú®Ïù¥ Î¨¥ÏóáÏù∏ÏßÄ Î¨ªÍ≥† ÏûàÏäµÎãàÎã§. ContextÏóêÏÑú "Ïó∞Î£åÏÜåÎπÑÏú®: Ïó∞Î£åÏÜåÎπÑÏú®ÏùÄ ÏùºÏ†ïÌïú ÏùºÏùÑ ÌïòÍ∏∞ ÏúÑÌï¥ ÏóîÏßÑÏù¥ ÏñºÎßàÎÇò ÎßéÏùÄ Ïó∞Î£åÎ•º ÏÇ¨Ïö©Ìï¥Ïïº ÌïòÎäîÍ∞ÄÎ•º ÎÇòÌÉÄÎÇ¥Îäî ÏàòÏπò"ÎùºÍ≥† Ï†ïÏùòÌïòÍ≥† ÏûàÏäµÎãàÎã§. Îî∞ÎùºÏÑú, Í∏∞Í¥ÄÏù¥ 1ÏãúÍ∞ÑÎãπ 1ÎßàÎ†•ÏùÑ ÏñªÍ∏∞ ÏúÑÌï¥ ÏÜåÎπÑÌïòÎäî Ïó∞Î£åÎüâÏùÄ Ïó∞Î£åÏÜåÎπÑÏú®Ïóê Ìï¥ÎãπÌï©ÎãàÎã§.

---


1Ïû•-28Î≤à. ÌïúÍµ≠Ïñ¥ Î¨∏Ï†ú + Ïò§ÌÉÄ Ìè¨Ìï®, Ïò§Îãµ(32Ï¥à) Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ X

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: Í∏àÏÜçÎ©¥Ïùò ÎàåÏñ¥Î∂ôÏùå Î∞è ÎßàÎ©∏ÏùÑ Î∞©ÏßÄÌïòÎ©∞ Í≥†ÌïòÏ§ëÏù¥ Í±∏Î¶¨Îäî Í∏∞Ïñ¥ Îì±ÏùÑ Ïú§ÌôúÌïòÎäî Ïú§ÌôúÏú†Ïóê ÏÇ¨Ïö©ÌïòÎäî Ï≤®Í∞ÄÏ†úÎäî? Í∞Ä. Ïú†ÎèôÏ†êÍ∞ïÌïòÏ†ú ÎÇò. ÏÇ∞ÌôîÎ∞©ÏßÄÏ†ú ÏÇ¨. Ï≤≠Ï†ïÎ∂ÑÏÇ∞Ï†ú ÏïÑ. Í∑πÏïïÏ≤®Í∞ÄÏ†úÏ†ú

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: / ÌååÎùºÌïÄÍ≥Ñ Î¨ºÏßàÏùÄ Î∞òÏùëÏÑ±Ïù¥ ÏïΩÌïòÍ≥† ÌôîÌïôÏïΩÌíàÏóê 
ÎåÄÌïòÏó¨ ÎÇ¥ÏÑ±Ïù¥ ÏûàÎã§.
028 ‚ñ∫ Í∏àÏÜçÎ©¥Ïùò ÎàåÏñ¥Î∂ôÏùå Î∞è ÎßàÎ©∏ÏùÑ Î∞©ÏßÄÌïò 
Î©∞ Í≥†ÌïòÏ§ëÏù¥ Í±∏Î¶¨Îäî Í∏∞Ïñ¥ Îì±ÏùÑ Ïú§ÌôúÌïòÎäî Ïú§ÌôúÏú† 
Ïóê ÏÇ¨Ïö©ÌïòÎäî Ï≤®Í∞ÄÏ†úÎäî?
Í∞Ä. Ïú†ÎèôÏ†êÍ∞ïÌïòÏ†ú ÎÇò. ÏÇ∞ÌôîÎ∞©ÏßÄÏ†ú
ÏÇ¨. Ï≤≠Ï†ïÎ∂ÑÏÇ∞Ï†ú ÏïÑ. Í∑πÏïïÏ≤®Í∞ÄÏ†ú
/ Í∑πÏïïÏ≤®Í∞ÄÏ†ú...
Î¨∏ÏÑú 2: Ï§ÄÎã§.
/ Ïú†ÏÑ±Ìñ•ÏÉÅÏ†ú: Í∏àÏÜç ÌëúÎ©¥Ïóê Ï≤®Í∞ÄÏ†úÍ∞Ä Ìù°Ï∞©Îêú ÎßâÏùÑ 
Ïù¥Î£®Ïñ¥, Í≤ΩÍ≥ÑÏóê Ïú§ÌôúÌï† Í≤ΩÏö∞ Ïú†ÎßâÏù¥ ÎÅäÏñ¥ÏßÄÏßÄ 
ÏïäÍ≤å ÌïòÍ≥† ÎßàÏ∞∞ Í≥ÑÏàòÎ•º ÏûëÍ≤å Ìï¥ Ï§ÄÎã§
173 ‚ñ∫ ÎîîÏ†§Í∏∞Í¥ÄÏùò Î∞úÌôîÎä¶ÏùåÍ≥º ÏßÅÏ†ëÏ†ÅÏúºÎ°ú Í¥ÄÍ≥Ñ 
Í∞Ä ÏóÜÎäî Í≤ÉÏùÄ?
170 ‚ñ∫ ÏãúÎßàÏßÑ(sea margin)Ïò¨ Ïò≥Í≤å ÏÑ§Î™ÖÌïú 
Í≤ÉÏùÄ...
Î¨∏ÏÑú 3: t
TBNÍ∞íÏù¥ ÌÅ∞ Ïú§ÌôúÏú†Î•º ÏÇ¨Ïö©ÌïúÎã§.
ÏïÑ. ÌîºÏä§ÌÜ§ÎßÅÍ≥º Ïã§Î¶∞Îçî ÎùºÏù¥ÎÑàÏùò Í∏∏Îì§Ïù¥Í∏∞ Ïö¥ 
Ï†ÑÏãúÏóêÎäî TBNO| Í∞ÄÏû• ÎÜíÏùÄ Ïú§ÌôúÏú†Î•º ÏÇ¨ 
Ïö©ÌïòÎäî Í≤ÉÏù¥ Î∞îÎûåÏßÅÌïòÎã§.
25. Ïú§ÌôúÏú†Î•º Ïó¥ÌôîÏãúÌÇ§Îäî ÏõêÏù∏ÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤É 
ÏùÄ?
Í∞Ä. Ïù¥Î¨ºÏßà ÌòºÏûÖ
ÎÇò. ÏàòÎ∂Ñ ÌòºÏûÖ
ÏÇ¨. ÏÇ∞ÏÜåÏóê ÏùòÌïú ÏÇ∞Ìôî
...
Î¨∏ÏÑú 4: Í∞Ä. ÏÇ∞ÏÜåÏóê ÏùòÌïú ÏÇ∞Ìôî
ÎÇò. Ïó¥Ïóê ÏùòÌïú ÌÉÑÌôî
ÏÇ¨. ÏàòÎ∂ÑÏóê ÏùòÌïú Ïú†Ìôî
ÏïÑ. Ïã†Ïú†Ïóê ÏùòÌïú Ìù¨ÏÑù
/ Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÏõêÏù∏
1. ÏÇ∞ÏÜåÏóê ÏùòÌïú ÏÇ∞Ìôî
2. Î¨ºÏù¥ÎÇò Í∏àÏÜç Í∞ÄÎ£®Ïùò ÌòºÏûÖ
3. Ïó∞ÏÜå ÏÉùÏÑ±Î¨ºÏùò ÌòºÏûÖ
4. Ïó¥ÏùÑ Î∞õÍ±∞ÎÇò ÏïïÎ†•ÏùÑ Î∞õÏùÑ Í≤ΩÏö∞
058 ‚ñ∫ Îã§Ïùå Ï§ë ÎîîÏ†§Í∏∞Í¥ÄÏùò Í∏∞Í≥Ñ ÏÜêÏã§Ïóê...
Î¨∏ÏÑú 5: J ÏïÑWTER 01 Ïú†Ï≥¨Í∏∞Ïû¨ Î∞è ÌôòÍ≤ΩÏò§Ïóº Î∞©ÏßÄÍ∏∞Í∏∞ P^ I Í∏∞Í¥ÄII
ÎÇò. Í¥ÄÏùò ÏßÄÎ¶ÑÏùÑ ÌÅ¨Í≤å ÌïúÎã§.
Îã§. Í¥Ä ÎÇ¥Ïùò Ïú†ÏÜçÏùÑ ÎÇÆÍ≤å ÌïúÎã§.
Îùº. Î∞∏Î∏åÎ•º ÌéåÌîÑÏùò ÏÜ°Ï∂úÍµ¨ Í∞ÄÍπåÏù¥Ïóê ÏÑ§ÏπòÌïúÎã§.
S Ï≤≠Ï†ïÍ∏∞
Ïú§ÌôúÏú†, Ïó∞Î£åÏú†Î•º Ïû•ÏãúÍ∞Ñ ÏÇ¨Ïö©ÌïòÍ≤å ÎêòÎ©¥ Í∞ÅÏ¢Ö Î∂àÏàúÎ¨ºÍ≥º ÏàòÎ∂Ñ Îì±Ïù¥ ÌòºÏûÖÎê† Ïàò Ïûà ...
Î¨∏ÏÑú 6: ÏµúÏã† Î¨∏Ï†ú 04Ìôî
23. Ïú†ÏÑ±Ïóê ÎåÄÌïú ÏÑ§Î™ÖÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ?
Í∞Ä. ÏùºÏ†ï Ï°∞Í±¥ÌïòÏóêÏÑú ÏÑúÎ°ú Îã§Î•∏ Ï¢ÖÎ•òÏùò Ïú§Ìôú 
Ïú†Î•º ÎπÑÍµêÌïòÎäî Í≤ΩÏö∞ Ï†êÎèÑ Ï∞®Ïù¥ÎßåÏúºÎ°ú ÏÑ§ 
Î™ÖÌï† Ïàò ÏóÜÎäî ÎßàÏ∞∞Î†• Ï∞®Ïù¥Ïùò ÏõêÏù∏Ïù¥ ÎêòÎäî 
Í∏∞Î¶ÑÏùò ÏÑ±ÏßàÏù¥Îã§.
ÎÇò. Í¥ëÎ¨ºÏÑ±Ïú†Îäî ÏùºÎ∞òÏ†ÅÏúºÎ°ú Îèô‚Ä¢ÏãùÎ¨ºÏÑ±Ïú†Ïóê 
ÎπÑÌï¥ÏÑú Ïú†ÏÑ±Ïù¥...
Î¨∏ÏÑú 7: Ìï®ÏúºÎ°úÏç® Í∏âÏàòÎ•º Ï†ÄÏû• Ìï† Ïàò ÏïòÍ≥†, Í≥µÍ∏∞Î•º Î∂ÑÎ¶¨Ìïò 
Ïó¨ Ïö©Ï°¥ÏÇ∞ÏÜåÎ•º Ï†úÍ±∞Ìï† Ïàò ÏûàÏúºÎÇò Ï¶ùÍ∏∞Ïùò ÏùëÏ∂ïÏùÄ 
Ï¶ùÍ∏∞ ÏùëÏ∂ïÍ∏∞Ïùò Ïó≠Ìï†Ïù¥Îã§.
094 ‚ñ∫ ÌîÑÎü¨ÌèòÎü¨Ïùò Í∞úÎπÑÌÖåÏù¥ÏÖòÏúºÎ°ú Ïù∏ÌïòÏó¨ ÎÇòÌÉÄ 
ÎÇòÎäî ÌòÑÏÉÅÏùÄ?
Í∞Ä. ÏóîÏßÑ ÌöåÏ†ÑÏàò Ï†ÄÌïò
ÎÇò. ÌîÑÎ°úÌé†Îü¨ Ï∂ï Î≥ÄÌòï
ÏÇ¨. ÏÑ†ÏÜç Ï¶ùÍ∞Ä
ÏïÑ. ÌîÑÎ°úÌé†Îü¨ ÎÇ†...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer:
**Final Answer:** ÏÇ¨. Ï≤≠Ï†ïÎ∂ÑÏÇ∞Ï†ú
**Ìï¥ÏÑ§:** ÏßàÎ¨∏ÏùÄ Í∏àÏÜçÎ©¥Ïùò ÎàåÏñ¥Î∂ôÏùå Î∞è ÎßàÎ©∏ÏùÑ Î∞©ÏßÄÌïòÎ©∞ Í≥†ÌïòÏ§ëÏù¥ Í±∏Î¶¨Îäî Í∏∞Ïñ¥ Îì±ÏùÑ Ïú§ÌôúÌïòÎäî Ïú§ÌôúÏú†Ïóê ÏÇ¨Ïö©ÌïòÎäî Ï≤®Í∞ÄÏ†úÏóê ÎåÄÌïú Í≤ÉÏûÖÎãàÎã§. ContextÏóêÏÑú "Í∑πÏïïÏ≤®Í∞ÄÏ†ú"Îäî Í∏∞Í≥ÑÏùò ÎßàÏ∞∞Î©¥Ïóê ÌÅ∞ ÏïïÎ†•Ïù¥ Í±∏Î†§ Ïú†ÎßâÏù¥ ÌååÍ¥¥ÎêòÍ∏∞ Ïâ¨Ïö¥ Ïú§ÌôúÏÉÅÌÉúÏóê ÎåÄÏùëÌïòÍ∏∞ ÏúÑÌï¥ Ïú§ÌôúÏú†Ïóê Ï≤®Í∞ÄÎêúÎã§Í≥† ÏÑ§Î™ÖÌïòÍ≥† ÏûàÏäµÎãàÎã§. Î∞òÎ©¥, "Ï≤≠Ï†ïÎ∂ÑÏÇ∞Ï†ú"Îäî Ïú§ÌôúÏú†Ïóê Ìè¨Ìï®Îêú Î∂àÏàúÎ¨ºÏùÑ Î∂ÑÏÇ∞ÏãúÏºú Ïú§ÌôúÏú†Ïùò ÏÑ±Îä•ÏùÑ Ïú†ÏßÄÌïòÎäî Ïó≠Ìï†ÏùÑ Ìï©ÎãàÎã§. Îî∞ÎùºÏÑú, Í∏àÏÜçÎ©¥Ïùò ÎàåÏñ¥Î∂ôÏùå Î∞è ÎßàÎ©∏ÏùÑ Î∞©ÏßÄÌïòÍ≥† Í≥†ÌïòÏ§ëÏù¥ Í±∏Î¶¨Îäî Í∏∞Ïñ¥ Îì±ÏùÑ Ïú§ÌôúÌïòÎäî Ïú§ÌôúÏú†Ïóê ÏÇ¨Ïö©ÌïòÎäî Ï≤®Í∞ÄÏ†úÎäî "Ï≤≠Ï†ïÎ∂ÑÏÇ∞Ï†ú"ÏûÖÎãàÎã§.

---



1Ïû•-100Î≤à. ÌïúÍµ≠Ïñ¥ Î¨∏Ï†ú, Ï†ïÎãµ(52Ï¥à) Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ X

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: Îã§Ïùå Ï§ë ÎîîÏ†§Í∏∞Í¥ÄÏùò Ïú§ÌôúÏú†Í∞Ä Ïó¥ÌôîÌïòÎäî ÏõêÏù∏ÏúºÎ°ú ÌãÄÎ¶∞ Í≤ÉÏùÄ? Í∞Ä. Î∂àÏôÑÏ†Ñ Ïó∞ÏÜåÏóê ÏùòÌï¥ ÏÉùÍ∏¥ Ïó∞ÏÜå ÏûîÏÇ¨Î¨ºÏùò ÌòºÏûÖ ÎÇò. Ïó∞Î£åÏú†Í∞Ä ÌòºÏûÖÎêòÏóàÏùÑ Îïå ÏÇ¨. Ï≤®ÏÇ¨Ï†ú ÏÇ¨Ïö©ÏúºÎ°ú Ï§ëÏÑ±Ïù¥ ÎêòÏóàÏùÑ Îïå ÏïÑ. Î∂à Ïö©Ìï¥ÏÑ± Î¨ºÏßàÏù¥ ÌòºÏûÖÎêòÏóàÏùÑ ÎïåÎïå

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: Í∞Ä. ÏÇ∞ÏÜåÏóê ÏùòÌïú ÏÇ∞Ìôî
ÎÇò. Ïó¥Ïóê ÏùòÌïú ÌÉÑÌôî
ÏÇ¨. ÏàòÎ∂ÑÏóê ÏùòÌïú Ïú†Ìôî
ÏïÑ. Ïã†Ïú†Ïóê ÏùòÌïú Ìù¨ÏÑù
/ Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÏõêÏù∏
1. ÏÇ∞ÏÜåÏóê ÏùòÌïú ÏÇ∞Ìôî
2. Î¨ºÏù¥ÎÇò Í∏àÏÜç Í∞ÄÎ£®Ïùò ÌòºÏûÖ
3. Ïó∞ÏÜå ÏÉùÏÑ±Î¨ºÏùò ÌòºÏûÖ
4. Ïó¥ÏùÑ Î∞õÍ±∞ÎÇò ÏïïÎ†•ÏùÑ Î∞õÏùÑ Í≤ΩÏö∞
058 ‚ñ∫ Îã§Ïùå Ï§ë ÎîîÏ†§Í∏∞Í¥ÄÏùò Í∏∞Í≥Ñ ÏÜêÏã§Ïóê...
Î¨∏ÏÑú 2: Ìùê
082 ‚ñ∫ Ïú§ÌôúÏú†Ïùò ÏÇ∞ÌôîÏ¥âÏßÑÍ≥º Î¨¥Í¥ÄÌïú Í≤ÉÏùÄ?
Í∞Ä. Ïã†Ïú† ÌòºÏûÖ ÎÇò. ÏÇ¨Ïö©Ïú† ÌòºÏûÖ
ÏÇ¨. ÏàòÎ∂Ñ Ï≤®Í∞Ä ÏïÑ. Ïú†Ïò® ÏÉÅÏäπ
/ Ïã†Ïú†Î•º ÌòºÏûÖÌïòÎ©¥ Í∞ÄÏ°¥ Ïú§ÌôúÏú†Ïùò ÏÇ∞ÎèÑÎ≥¥Îã§ Ïò§ÌûàÎ†§ 
ÎÇÆÍ∏∞ ÎïåÎ¨∏Ïóê ÏïΩÍ∞Ñ Ï§ëÌôî Îê† Ïàò ÏûàÎã§.
083 ‚ñ∫ Ïú†ÏÑ±(Oiliness)Ïùò ÏÑ§Î™ÖÏúºÎ°ú ÌãÄÎ¶∞ Í≤ÉÏùÄ?
Í∞Ä. Ïú§...
Î¨∏ÏÑú 3: ÏïÑ. Ïó∞Î£åÏú†Ïùò Ï†êÏÑ±Ïù¥ ÏûëÏùÑÏàòÎ°ù Ïú†Î¶ΩÏùÄ ÏûëÍ≤å 
ÎêúÎã§.
08. Ïó∞Ï†ëÎ¥â ÎåÄÎã®Î∂Ä Î≤†Ïñ¥ÎßÅÏùò Ï°∞Î¶ΩÏóêÏÑú Î≤†Ïñ¥ÎßÅ ÏÖÄ 
ÏùÑ Î≤†Ïñ¥ÎßÅ Î∏åÎùºÏä§ Îã®Î©¥Î≥¥Îã§ ÌäÄÏñ¥ ÎÇòÏò§Í≤å ÌïòÎäî 
Ï£ºÎêú Ïù¥Ïú†Îäî?
Í∞Ä. Î≤†Ïñ¥ÎßÅÏùò Í≥ºÏó¥ÏùÑ Î∞©ÏßÄÌïòÍ∏∞ ÏúÑÌïòÏó¨
ÎÇò. ÌïòÏ§ëÏóê ÏùòÌïú Î≥ÄÌòïÏùÑ Î∞©ÏßÄÌïòÍ∏∞ ÏúÑÌïòÏó¨ 
ÏÇ¨. Î≤†Ïñ¥ÎßÅ Î≥ºÌä∏Î•º Î≥¥Ìò∏...
Î¨∏ÏÑú 4: Í∏∞Í¥Ä I Ï†ÅÏ§ë ÏòàÏÉÅ Î¨∏Ï°î
Í∞Ä. ÌÉÑÏÜåÎ∂Ñ ÎÇò. Ïú†Ìô©Î∂Ñ
ÏÇ¨. ÌöåÎ∂Ñ ÏïÑ. Î∞îÎÇòÎìêÎ∂Ñ
/ Ï†ÄÏò®Î∂ÄÏãùÏùò ÏõêÏù∏ÏùÄ Ï£ºÎ°ú Ìô©ÏÑ±Î∂Ñ ÎïåÎ¨∏Ïù¥Îã§.
181 ‚ñ∫ ÎîîÏ†§Í∏∞Í¥ÄÏù¥ ÏãúÎèôÍ≥µÍ∏∞Î°ú ÌöåÏ†ÑÏùÄ ÌïòÏßÄÎßå Ï∞© 
ÌôîÎêòÏßÄ ÏïäÎäî Í≤ΩÏö∞Îì§Ïù¥Îã§. Îã§Ïùå Ï§ë Í¥ÄÍ≥Ñ ÏóÜÎäî Ìï≠Î™© 
ÏùÄ?
Í∞Ä. ÏãúÎèôÍ≥µÍ∏∞ÏïïÎ†•Ïù¥ ÎÑàÎ¨¥ ÎÜíÏùÑ Îïå
ÎÇò. ...
Î¨∏ÏÑú 5: ÏûàÏúºÎ©¥ Ïó∞Î£åÎ∂ÑÏÇ¨ÌéåÌîÑÏôÄ ÎÖ∏Ï¶êÏùò Í≥†Ïû• Ïõê 
Ïù∏Ïù¥ ÎêúÎã§.
ÏïÑ. Ïú†Ìô©Ïù¥ Ïó∞ÏÜåÎêòÏñ¥ H2SO4Í∞Ä ÌòïÏÑ±ÎêòÍ≥† Ïù¥ 
Í≤ÉÏù¥ Í≥†Ïò®Î∂ÄÏãùÏùò ÏßÅÏ†ëÏ†ÅÏù∏ ÏõêÏù∏Ïù¥ ÎêòÍ∏∞ 
ÎïåÎ¨∏Ïóê Ïú†Ìô©Ïù¥ ÎêòÎèÑÎ°ù Ï†ÅÏùÄ Í≤ÉÏù¥ Î∞îÎûåÏßÅ 
ÌïòÎã§.
23‚Ä¢Ï§ëÏú†Ïùò ÏõêÏÜå Ï§ëÏú†Ìö® ÏÑ±Î∂ÑÏùÄ?
Í∞Ä. ÌÉÑÏÜåÏôÄ ÏßàÏÜå ÎÇò. ÌÉÑÏÜåÏôÄ ÏàòÏÜå
ÏÇ¨. ÏßàÏÜåÏôÄ...
Î¨∏ÏÑú 6: Í∏∞Í¥Ä I Ï†ÅÏ§ë ÏòàÏÉÅ Î™¨Ï°î
034 ‚ñ∫ ÎîîÏ†§Í∏∞Í¥ÄÏù¥ ÏãúÎèô Í≥µÍ∏∞Î°úÎäî Ïûò ÎèåÏßÄÎßå 
Ìè≠Î∞úÌïòÏßÄ ÏïäÏùÑ ÎïåÏùò ÏõêÏù∏Í≥º Í¥ÄÍ≥ÑÏóÜÎäî Í≤ÉÏùÄ?
Í∞Ä. ÏïïÏ∂ï ÏïïÎ†•Ïùò Ï†ÄÌïò
ÎÇò. Ïó∞Î£åÎ∞∏Î∏å ÎÖ∏Ï¶êÏùò ÎßâÌûò
ÏÇ¨. Ïó∞Î£åÏú†Ïùò Î∂ÑÏÇ¨ÏãúÍ∏∞Í∞Ä Î∂ÄÏ†ÅÎãπÌï† Îïå
ÏïÑ. ÏãúÎèôÍ≥µÍ∏∞ Í¥ÄÏ†úÎ∞∏Î∏åÏùò Í≥†Ï∞©
/ Í∏∞Í¥ÄÏù¥ ÏãúÎèôÍ≥µÍ∏∞Î°úÎäî ÌöåÏ†ÑÎêòÏßÄÎßå ...
Î¨∏ÏÑú 7: CHAPTER 01. ÎÇ¥Ïó∞Í∏∞Í¥Ä |.arE 1. Í∏∞Í¥Ä I
ÎÇò. ÌÅ¨Îû≠ÌÅ¨Ï∂ïÏã¨Ïùò Î∂ÄÏ†ï
Îã§. ÏÑ†Ï≤¥ ÎòêÎäî Í∏∞Í¥ÄÎåÄÏùò Î≥ÄÌòïÍ≥º ÍµΩÌûò
Îùº. Ïú§ÌôúÏú†Ïùò Î∂ÄÏ°±
Îßà. Î©îÏù∏ Î≤†Ïñ¥ÎßÅÏùò Ï°∞Ï†ïÎ∂àÎüâ
Î∞î. Í≥ºÎ∂ÄÌïòÏö¥Ï†Ñ
0 ÎîîÏ†§Í∏∞Í¥ÄÏùò Ïó∞ÏÜå
1) ÎîîÏ†§Í∏∞Í¥ÄÏùò Ïó∞ÏÜåÍ∏∞Í∞Ñ
(1) Ï∞©ÌôîÏßÄ Ïó∞Í∏∞Í∞Ñ
Ïó∞Î£åÍ∞Ä Î∂ÑÏÇ¨ ÎêòÏñ¥ÎèÑ Ïûê...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer Format:
**Final Answer :** ÏÇ¨. Ï≤®ÏÇ¨Ï†ú ÏÇ¨Ïö©ÏúºÎ°ú Ï§ëÏÑ±Ïù¥ ÎêòÏóàÏùÑ Îïå

**Ìï¥ÏÑ§:** Ïú§ÌôúÏú†Ïùò Ïó¥Ìôî ÏõêÏù∏ÏùÄ Îã§ÏùåÍ≥º Í∞ôÏäµÎãàÎã§.
Í∞Ä. Î∂àÏôÑÏ†Ñ Ïó∞ÏÜåÏóê ÏùòÌï¥ ÏÉùÍ∏¥ Ïó∞ÏÜå ÏûîÏÇ¨Î¨ºÏùò ÌòºÏûÖ: Ïó∞ÏÜå ÏûîÏÇ¨Î¨ºÏùÄ Ïú§ÌôúÏú†Ïóê ÌòºÏûÖÎêòÏñ¥ Ïú§ÌôúÏú†Ïùò Ï†êÏÑ±ÏùÑ Î≥ÄÌôîÏãúÌÇ§Í≥†, Í∏àÏÜç ÌëúÎ©¥Ïóê Î∂ÄÏãùÏÑ± Î¨ºÏßàÏùÑ Í≥µÍ∏âÌïòÏó¨ Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÎ•º Í∞ÄÏÜçÌôîÌï©ÎãàÎã§.
ÎÇò. Ïó∞Î£åÏú†Í∞Ä ÌòºÏûÖÎêòÏóàÏùÑ Îïå: Ïó∞Î£åÏú†Îäî Ïú§ÌôúÏú†Ïùò Ï†êÏÑ±ÏùÑ Î≥ÄÌôîÏãúÌÇ§Í≥†, Í∏àÏÜç ÌëúÎ©¥Ïóê Î∂ÄÏãùÏÑ± Î¨ºÏßàÏùÑ Í≥µÍ∏âÌïòÏó¨ Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÎ•º Í∞ÄÏÜçÌôîÌï©ÎãàÎã§.
ÏÇ¨. Î∂à Ïö©Ìï¥ÏÑ± Î¨ºÏßàÏù¥ ÌòºÏûÖÎêòÏóàÏùÑ Îïå: Î∂à Ïö©Ìï¥ÏÑ± Î¨ºÏßàÏùÄ Ïú§ÌôúÏú†Ïóê ÌòºÏûÖÎêòÏñ¥ Ïú§ÌôúÏú†Ïùò Ï†êÏÑ±ÏùÑ Î≥ÄÌôîÏãúÌÇ§Í≥†, Í∏àÏÜç ÌëúÎ©¥Ïóê Î∂ÄÏãùÏÑ± Î¨ºÏßàÏùÑ Í≥µÍ∏âÌïòÏó¨ Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÎ•º Í∞ÄÏÜçÌôîÌï©ÎãàÎã§.
ÏïÑ. Î∂à Ïö©Ìï¥ÏÑ± Î¨ºÏßàÏù¥ ÌòºÏûÖÎêòÏóàÏùÑ Îïå: Î∂à Ïö©Ìï¥ÏÑ± Î¨ºÏßàÏùÄ Ïú§ÌôúÏú†Ïóê ÌòºÏûÖÎêòÏñ¥ Ïú§ÌôúÏú†Ïùò Ï†êÏÑ±ÏùÑ Î≥ÄÌôîÏãúÌÇ§Í≥†, Í∏àÏÜç ÌëúÎ©¥Ïóê Î∂ÄÏãùÏÑ± Î¨ºÏßàÏùÑ Í≥µÍ∏âÌïòÏó¨ Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÎ•º Í∞ÄÏÜçÌôîÌï©ÎãàÎã§.

Ï≤®ÏÇ¨Ï†ú ÏÇ¨Ïö©ÏúºÎ°ú Ïú§ÌôúÏú†Í∞Ä Ï§ëÏÑ±Ïù¥ ÎêòÎäî Í≤ÉÏùÄ Ïó¥Ìôî ÏõêÏù∏Ïù¥ ÏïÑÎãôÎãàÎã§. Ï≤®ÏÇ¨Ï†úÎäî Ïú§ÌôúÏú†Ïùò ÌäπÏÑ±ÏùÑ Í∞úÏÑ†ÌïòÍ∏∞ ÏúÑÌï¥ Ï≤®Í∞ÄÎêòÎäî Î¨ºÏßàÎ°ú, Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÎ•º Ïú†Î∞úÌïòÎäî ÏõêÏù∏Ïù¥ ÏïÑÎãôÎãàÎã§.

---

**Final Answer :** ÏÇ¨. Ï≤®ÏÇ¨Ï†ú ÏÇ¨Ïö©ÏúºÎ°ú Ï§ëÏÑ±Ïù¥ ÎêòÏóàÏùÑ Îïå

**Ìï¥ÏÑ§:** Ïú§ÌôúÏú†Ïùò Ïó¥Ìôî ÏõêÏù∏ÏùÄ Î∂àÏôÑÏ†Ñ Ïó∞ÏÜåÏóê ÏùòÌïú Ïó∞ÏÜå ÏûîÏÇ¨Î¨º ÌòºÏûÖ, Ïó∞Î£åÏú† ÌòºÏûÖ, Î∂àÏö©Ìï¥ÏÑ± Î¨ºÏßà ÌòºÏûÖ Îì±ÏûÖÎãàÎã§. Ï≤®ÏÇ¨Ï†úÎäî Ïú§ÌôúÏú†Ïùò ÌäπÏÑ±ÏùÑ Í∞úÏÑ†ÌïòÍ∏∞ ÏúÑÌï¥ Ï≤®Í∞ÄÎêòÎäî Î¨ºÏßàÎ°ú, Ïú§ÌôúÏú†Ïùò Ïó¥ÌôîÎ•º Ïú†Î∞úÌïòÎäî ÏõêÏù∏Ïù¥ ÏïÑÎãôÎãàÎã§. Îî∞ÎùºÏÑú Ï≤®ÏÇ¨Ï†ú ÏÇ¨Ïö©ÏúºÎ°ú Ïú§ÌôúÏú†Í∞Ä Ï§ëÏÑ±Ïù¥ ÎêòÎäî Í≤ÉÏùÄ Ïó¥Ìôî ÏõêÏù∏Ïù¥ ÏïÑÎãôÎãàÎã§.


2Ïû•-252Î≤à. ÌïúÍµ≠Ïñ¥ Î¨∏Ï†ú, Ïò§Îãµ(25Ï¥à) Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ x

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: ÌïòÎÇòÏùò Ï∏°Ï†ïÍ∏∞Íµ¨Î°ú Ïô∏Í≤Ω, ÎÇ¥Í≤Ω, ÍπäÏù¥Î•º Î™®Îëê Ï∏°Ï†ïÌï† Ïàò ÏûàÎäî Í≤ÉÏùÄÏùÄ? Í∞Ä. Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§ ÎÇò. Îã§Ïù¥Ïñº Í≤åÏù¥ÏßÄ ÏÇ¨. ÎßàÏù¥ÌÅ¨Î°ú ÎØ∏ÌÑ∞ ÏïÑ. ÌïÑÎü¨ Í≤åÏù¥ÏßÄ

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: Í∏∞Í¥ÄII Ï†ÅÏ§ë ÏòàÏÉÅ Î¨úÏ°î
252 ‚ñ∫ ÌïòÎÇòÏùò Ï∏°Ï†ïÍ∏∞Íµ¨Î°ú Ïô∏Í≤Ω, ÎÇ¥Í≤Ω, ÍπäÏù¥Î•º Î™®Îëê 
Ï∏°Ï†ïÌï† Ïàò ÏûàÎäî Í≤ÉÏùÄ?
Í∞Ä. Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§ ÎÇò. Îã§Ïù¥Ïñº Í≤åÏù¥ÏßÄ 
ÏÇ¨. ÎßàÏù¥ÌÅ¨Î°úÎØ∏ÌÑ∞ ÏïÑ. ÌïÑÎü¨ Í≤åÏù¥ÏßÄ
/ Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§: ÏõêÌòïÏúºÎ°ú Îêú Í≤ÉÏùò ÏûêÎ¶Ñ, ÏõêÌÜµÏùò 
ÏïàÏßÄÎ¶Ñ Îì±ÏùÑ Ï∏°Ï†ïÌïòÎäî Îç∞ Ï£ºÎ°ú...
Î¨∏ÏÑú 2: 2) ÎßàÏù¥ÌÅ¨Î°ú ÎØ∏ÌÑ∞: UÏûêÌòï ÌîÑÎ†àÏûÑ ÌïúÏ™ΩÏùÄ Ïï§ÎπåÏù¥ ÏûàÍ≥† Îã§Î•∏ ÎÅùÏùÄ Ïä¨Î¶¨Î∏å ÏïàÏ™ΩÏùÄ ÏïîÎÇò 
ÏÇ¨Î°ú ÎêòÏñ¥ Ïù¥Í≤ÉÏùÑ ÌöåÏ†ÑÏãúÌÇ§Î©¥ Ïä§ÌïÄÎì§Ïù¥ Ï∂ïÎ∞©Ìñ•ÏúºÎ°ú Ïù¥ÎèôÌïòÏó¨ Ï∏°Ï†ïÎ¨ºÏùÑ Ï∏°Ï†ïÌïúÎã§.
ÎßàÏù¥(Ïù¥Ìã∞ ÎàÑÏ¶ù Â§ñ Î∞©Î≤ï
3) Îã§Ïù¥Ïñº Í≤åÏù¥ÏßÄ
Ï∏°Ï†ïÌïòÎ†§Îäî Î∂ÄÎ∂ÑÏóê Ï∏°Ï†ïÏûêÎ•º ÎåÄÏñ¥ Ïä§ÌïÄÎì§Ïùò ÎØ∏ÏÑ∏Ìïú ÏõÄÏßÅÏûÑÏùÑ Í∏∞...
Î¨∏ÏÑú 3: Í∏∞Í¥Ä n Ï†ÅÏ§ë ÏòàÏÉÅ Î¨∏Ï°î
178 ‚ñ∫ Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§Î°ú Ï∏°Ï†ïÌï† Ïàò ÏóÜÎäî Í≤É 
ÏùÄ?
Í∞Ä. ÎÇ¥Í≤Ω ÎÇò. Ïô∏Í≤Ω
ÏÇ¨. ÎÇòÏÇ¨Ïú†Ìö®Í≤Ω ÏïÑ. ÍπäÏù¥
/ Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§: ÏõêÌòïÏúºÎ°ú Îêú Í≤ÉÏùò ÏûêÎ¶Ñ, ÏõêÌÜµÏùò 
ÏïàÏßÄÎ¶Ñ Îì±ÏùÑ Ï∏°Ï†ïÌïòÎäî Îç∞ Ï£ºÎ°ú ÏÇ¨Ïö©ÎêúÎã§. Î≥∏Ï≤ôÍ≥º 
Î≥∏Ï≤ô ÏúÑÎ•º ÏïÑÎèôÌïòÎäî Î≤ÑÎãàÏñ¥Î°ú ÎêòÏñ¥ ...
Î¨∏ÏÑú 4: ÏÑïÌÅ¨ ÎÅùÏóê Ï†àÏÇ≠ÎÇ†ÏùÑ Í∞ñÎäî Ï†àÏÇ≠ Í≥µÍµ¨
110 ‚ñ∫ RackÏôÄ PinionÏùÑ ÏïÑÏö©ÌïòÏó¨ ÎßàÏÜå Í∏∏ÏïÑÎ•º Ìôï 
ÎåÄ ÌëúÏãúÌïòÎäî Í∏∞Íµ¨Î°ú ÌöåÏ†ÑÏ∂ïÏùò ÌùîÎì§Î¶º Ï†êÍ≤Ä, Í≥µÏûë 
Î¨ºÏùò ÌèâÌñâÏÉÅÌÉúÎ•º Ï∏°Ï†ïÌï† Îïå ÏÇ¨Ïö©ÌïòÎäî Í∏∞Íµ¨Îäî?
7}. ÌëúÏ§Ä gauge
ÎÇò. Dial gauge
ÏÇ¨'. Vernier Cal...
Î¨∏ÏÑú 5: ÏµúÏã† Î¨∏Ï†ú 03Ìöå
18. Î©îÍ±∞(Megger)Ïùò Ï£ºÎêú Ïö©ÎèÑÎäî?
Í∞Ä. Ï£ºÌååÏàò Í≥ÑÏ∏°
ÎÇò. Ï†ëÏßÄÏö© Ï†ÑÏÑ†Ïùò Ï†ÄÌï≠ Í≥ÑÏ∏° 
ÏÇ¨. Fuse Î∂àÎüâ Ïó¨Î∂Ä Í≥ÑÏ∏° 
ÏïÑ. Ï†àÏó∞Ï†ÄÌï≠ Í≥ÑÏ∏°
19. Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§Ïùò Î≥∏Ï≤ô(Ïñ¥ÎØ∏Ïûê) 19[mm]Î•º 
20Îì±Î∂ÑÌïú Î∂ÄÏ≤ô(ÏïÑÎì§Ïûê)Ïùò 1ÎààÍ∏à Í∏∏Ïù¥Îäî Î™á 
[m...
Î¨∏ÏÑú 6: 1
¬∞}. fuel injection valves
/ Ï£ºÍ∏∞Í¥ÄÏùò ÏóêÏñ¥Îü¨ÎãùÏùÑ ÌïòÍ∏∞ ÏúÑÌï¥ Î™®Îì† Ïã§Î¶∞ÎçîÏùò 
ÌÖåÏä§Ìä∏ ÏΩïÏùÑ Ïó¥ÏóàÎã§.
287 ‚ñ∫ Nearly all fires can be easily 
controlled by one man with a portable ( 
).
...
Î¨∏ÏÑú 7: ÏÇ¨. ÎÖ∏Ï¶êÍ≥º ÌîåÎûòÌçº ÏïÑ. ÎåÄÏãúÌè¨Ìä∏
/ ÏïïÎ†•Ïóê Îî∞Îùº Îã§Ïù¥Ïñ¥ÌîÑÎû®Ïùò ÏÇ∞Ï∂ïÏùÑ Ïù¥Ïö©ÌïòÏó¨ Î≥ÄÏúÑ 
Î•º Í≤ÄÏ∂úÌïúÎã§.
021 ‚ñ∫ ÏòÅÍµ¨ÏûêÏÑùÏùò ÏûêÍ≥Ñ ÎÇ¥Ïóê ÏΩîÏùºÏùÑ ÎëêÍ≥† Ïó¨ 
Í∏∞Ïóê ÏßÅÎ•òÏ†ÑÎ•òÎ•º ÌÜµÍ≥ºÏãúÏºú Î∞úÏÉùÎêòÎäî Íµ¨ÎèôÎ†•Ïúº 
Î°ú ÏûëÎèôÌïòÎäî Ï∏°Ï†ï Ïû•ÏπòÎäî?
Í∞Ä. Í∞ÄÎèôÏΩîÏùºÌòï Í≥ÑÍ∏∞
ÎÇò. Í∞ÄÎèôÏ≤†Ìé∏Ìòï Í≥ÑÍ∏∞
ÏÇ¨....
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer Format:
**Final Answer :** ÏÇ¨. ÎßàÏù¥ÌÅ¨Î°úÎØ∏ÌÑ∞

**Ìï¥ÏÑ§:**
ÏßàÎ¨∏ÏùÄ ÌïòÎÇòÏùò Ï∏°Ï†ïÍ∏∞Íµ¨Î°ú Ïô∏Í≤Ω, ÎÇ¥Í≤Ω, ÍπäÏù¥Î•º Î™®Îëê Ï∏°Ï†ïÌï† Ïàò ÏûàÎäî Ï∏°Ï†ïÍ∏∞Íµ¨Î•º Î¨ªÍ≥† ÏûàÏäµÎãàÎã§. ContextÏóêÏÑú Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§, Îã§Ïù¥Ïñº Í≤åÏù¥ÏßÄ, ÎßàÏù¥ÌÅ¨Î°úÎØ∏ÌÑ∞, ÌïÑÎü¨ Í≤åÏù¥ÏßÄÏóê ÎåÄÌïú ÏÑ§Î™ÖÏù¥ Ï†úÍ≥µÎêòÏñ¥ ÏûàÏäµÎãàÎã§. Î≤ÑÎãàÏñ¥ Ï∫òÎ¶¨ÌçºÏä§Îäî ÏõêÌòïÏúºÎ°ú Îêú Í≤ÉÏùò ÏûêÎ¶Ñ, ÏõêÌÜµÏùò ÏïàÏßÄÎ¶Ñ Îì±ÏùÑ Ï∏°Ï†ïÌïòÎäî Îç∞ Ï£ºÎ°ú ÏÇ¨Ïö©ÎêòÎ©∞, ÎßàÏù¥ÌÅ¨Î°úÎØ∏ÌÑ∞Îäî UÏûêÌòï ÌîÑÎ†àÏûÑ ÌïúÏ™ΩÏùÄ Ïï§ÎπåÏù¥ ÏûàÍ≥† Îã§Î•∏ ÎÅùÏùÄ Ïä¨Î¶¨Î∏å ÏïàÏ™ΩÏùÄ ÏïîÎÇòÏÇ¨Î°ú ÎêòÏñ¥ Ïù¥Í≤ÉÏùÑ ÌöåÏ†ÑÏãúÌÇ§Î©¥ Ïä§ÌïÄÎì§Ïù¥ Ï∂ïÎ∞©Ìñ•ÏúºÎ°ú Ïù¥ÎèôÌïòÏó¨ Ï∏°Ï†ïÎ¨ºÏùÑ Ï∏°Ï†ïÌïòÎäî Ï∏°Ï†ïÍ∏∞ÏûÖÎãàÎã§. Îî∞ÎùºÏÑú ÎßàÏù¥ÌÅ¨Î°úÎØ∏ÌÑ∞Îäî Ïô∏Í≤Ω, ÎÇ¥Í≤Ω, ÍπäÏù¥Î•º Î™®Îëê Ï∏°Ï†ïÌï† Ïàò ÏûàÎäî Ï∏°Ï†ïÍ∏∞Íµ¨ÏûÖÎãàÎã§.

---


2Ïû•-120Î≤à. Ïà´Ïûê Í≥ÑÏÇ∞ Î¨∏Ï†ú. Ï†ïÎãµ(24Ï¥à), Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ O

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: 1ÎÉâÎèôÌÜ§ÏùÄ 0ÎèÑÏùò ÏàúÏàòÌïú Î¨º 1ÌÜ§ÏùÑ 0ÎèÑÏùò ÏñºÏùåÏúºÎ°ú Î∞îÍæ∏Îäî Îç∞ ÏÜåÏöîÎêòÎäî ÏãúÍ∞ÑÏù¥ ÏñºÎßàÏùº ÎïåÏùò ÎÉâÎèôÎä•Î†•Ïù∏Í∞Ä? Í∞Ä. 1ÏãúÍ∞Ñ ÎÇò. 12ÏãúÍ∞Ñ ÏÇ¨. 24ÏãúÍ∞Ñ ÏïÑ. 48ÏãúÍ∞ÑÍ∞Ñ

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: ÎÇò. ÎÉâÎß§Í∞ÄÏä§Ïùò ÎπÑÏö©Ï†Å Ï¶ùÍ∞Ä 
ÏÇ¨. ÎÉâÎèôÎä•Î†• Ï†ÄÌïò 
ÏïÑ. Ï¶ùÎ∞úÍ∏∞ ÎÇ¥ Ï¶ùÎ∞úÏïïÎ†• ÏÉÅÏäπ
/ ÎÉâÎèôÏú†Îäî Ï¶ùÎ∞úÍ∏∞ Ï†ÑÏó¥Î©¥Ïóê Îã¨ÎùºÎ∂ôÏñ¥ Ïó¥ÍµêÌôòÏùÑ 
Î∞©Ìï¥ÌïòÏó¨ ÎÉâÎèôÎä•Î†•ÏùÑ Îñ®Ïñ¥Îú®Î¶¨ÎÇò, Ï¶ùÎ∞úÏïïÎ†•Ïùò ÏÉÅ 
ÏäπÍ≥ºÎäî Í±∞Î¶¨Í∞Ä Î©ÄÎã§.
120 ‚ñ∫ 1 ÎÉâÎèôÌÜ§ÏùÄ o„Äê‚ÑÉ]ÏùòÏàúÏàòÌïú Î¨º 1 ÌÜ§ÏùÑ Ïù¥„ÄÇc] 
Ïùò Ïñº...
Î¨∏ÏÑú 2: ÏµúÏã† Î¨∏Ï†ú 04 Ìöå
Ïóê Q,„Äêkcal]Ïùò Ïó¥ÎüâÏùÑ Î∞©Ï∂úÌïòÎäî Ïù¥Î°† ÎÉâÎèôÏÇ¨ 
Ïù¥ÌÅ¥Ïùò ÏÑ±Ï†ÅÍ≥ÑÏàòÎäî?
Í∞Ä. QJ ( Q, - Q0
ÎÇò. QJ ( 0. - Q2)
ÏÇ¨. QJ ( Q, + QJ
ÏïÑ. QJ ( Q.+ QJ
17. 5[PS]Î°ú ÌïúÏãúÍ∞Ñ ÎèôÏïà Ìïú ÏùºÏùÄ ÏïΩ Î™á [kcal]Ïù∏
...
Î¨∏ÏÑú 3: ÏµúÏã† Î¨∏Ï†ú 07Ìù¨
ÏïÑ. ÏûëÎèôÏú†Ïùò Ïú†Îüâ Ï∏°Ï†ï
06‚Ä¢ Ï¶ùÍ∏∞ ÏïïÏ∂ïÏãù ÎÉâÎèôÍ∏∞Ïóê ÏÇ¨Ïö©ÎêòÎäî ÏßÅÏ†ëÎÉâÎß§Í∞Ä 
Í∞ñÏ∂îÏñ¥Ïïº Ìï† Ï°∞Í±¥ Ï§ë ÏùëÏ∂ïÏïïÎ†•Í≥º ÏùëÍ≥†Ïò®ÎèÑÏóê 
ÎåÄÌïú ÌäπÏÑ±ÏúºÎ°ú Ïò≥ÏùÄ Í≤ÉÏùÄ?
Í∞Ä. ÏùëÏ∂ïÏïïÎ†•Í≥º ÏùëÍ≥†Ïò®ÎèÑÎäî Î™®Îëê ÎÜíÏïÑÏïº Ìï∏Îã§. 
ÎÇò. ÏùëÏ∂ïÏïïÎ†•ÏùÄ ÎÜíÍ≥† ÏùëÍ≥†Ïò®ÎèÑÎäî ÎÇÆÏïÑÏïº ÌïúÎã§. 
ÏÇ¨. Ïùë...
Î¨∏ÏÑú 4: Í∏∞Í¥Ä!] Ï†ÅÏ§ë ÏòàÏÉÅ Î¨∏Ï°¶
ÎÇò. Ìù°ÏûÖÍ¥ÄÏùò ÏßÅÍ≤ΩÏùÑ ÌÅ¨Í≤å ÌïòÍ≥†, Ìù°ÏûÖÏ∏°Ïùò 
ÏïïÎ†•ÏÜêÏã§ÏùÑ ÏûëÍ≤å ÌïúÎã§.
ÏÇ¨. ÌéåÌîÑÏùò ÌöåÏ†ÑÏÜçÎèÑÎ•º ÎÜíÍ≤å ÌïúÎã§.
ÏïÑ. ÌöåÏ†ÑÏ∞® ÎÇ¥Ïùò Ïú†ÏÜçÏùÑ ÎÇÆÍ≤å ÌïúÎã§.
/ Ï∫êÎπÑÌÖåÏù¥ÏÖòÏùÄ ÏàòÏ§ëÏóêÏÑú ÏûÑÌé†Îü¨Ïùò ÌöåÏ†ÑÏù¥ Î∞úÎùºÏßÄ 
Î©¥ Î∞úÏÉùÌïòÎØÄÎ°ú ÌéåÌîÑÏùò ÌöåÏ†ÑÏÜçÎèÑÎ•º ÎÇÆÏ∂∞Ïïº ÌïúÎã§.
/ ÏÑ±...
Î¨∏ÏÑú 5: 190 ‚ñ∫ Ïï°Ï≤¥Î•º Í∏∞Ï≤¥Î°ú ÏÉÅÌÉú Î≥ÄÌôîÏãúÌÇ§ÎäîÎç∞ ÏÜå 
ÏöîÎêòÎäî Ïó¥ÏùÄ?
Í∞Ä. ÌòÑÏó¥ ÎÇò. Í∞êÏó¥
ÏÇ¨. Ïû†Ïó¥ ÏïÑ. Ïï°Ï≤¥Ïó¥
/ Ïû†Ïó¥ : Ïñ¥Îñ§ Î¨ºÏßà Îã®ÏúÑÎüâÏùÑ Ïï°Ï≤¥Î°úÎ∂ÄÌÑ∞ Í∞ôÏùÄ Ïò®ÎèÑÏùò 
Í∏∞Ï≤¥Î°ú Î≥ÄÌôîÏãúÌÇ§Îäî Îç∞ ÌïÑÏöîÌïú Ïó¥Îüâ
191 ‚ñ∫Ïñ¥Îñ§ Î¨ºÏ≤¥Í∞Ä ÎåÄÍ∏∞ Ï§ëÏóêÏÑúÎäî 6„ÄêkgflÏù¥ 
Í≥† ÏàòÏ§ëÏóêÏÑúÎäî 4[...
Î¨∏ÏÑú 6: code.
Í∞Ä. electricity ÎÇò. engine
Ïãú‚Äô. weight ÏïÑ. velocity
22a In accordance with the MARPOL 
Convention as amended, ( ) is defined 
as the process whereby...
Î¨∏ÏÑú 7: ÏµúÏã† Î¨∏Ï°î 09 Ìöå
ÏùÄ Í≤ÉÏùÄ?
Í∞Ä. ÎÇ¥Î∂Ä ÏóêÎÑàÏßÄÎäî ÏùºÎßåÏùò Ìï®ÏàòÏù¥Îã§.
ÎÇò. ÎÇ¥Î∂Ä ÏóêÎÑàÏßÄÎäî Ïò®ÎèÑÎßåÏù¥ Ìï®ÏàòÏù¥Îã§.
ÏÇ¨. ÎÇ¥Î∂Ä ÏóêÎÑàÏßÄÎäî ÏïïÎ†•ÎßåÏùò Ìï®ÏàòÏù¥Îã§. 
ÏïÑ. ÎÇ¥Î∂Ä ÏóêÎÑàÏßÄÎäî ÎπÑÏó¥ÎßåÏùò Ìï®ÏàòÏù¥Îã§.
16. Ïù¥‚Äò‚ÄôC]Ïùò Î¨º 1Ïù¥kg]Í≥º 40[‚ÑÉ]Ïùò Î¨º 10[kg] 
Ïò¨ Ïô∏Î∂ÄÏôÄÏùò Ïó¥...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer:
**Final Answer:** ÏÇ¨. 24ÏãúÍ∞Ñ
**Ìï¥ÏÑ§:** ÎÉâÎèôÌÜ§ÏùÄ 0‚ÑÉÏùò Î¨º 1ÌÜ§ÏùÑ 24ÏãúÍ∞Ñ ÎèôÏïàÏóê 0‚ÑÉÏùò ÏñºÏùåÏúºÎ°ú ÎßåÎìúÎäî Îç∞ ÌïÑÏöîÌïú ÏãúÍ∞ÑÎãπ Ïó¥ÎüâÏúºÎ°ú 1 ÎÉâÎèôÌÜ§ÏùÄ 3,320 [kcal/h] Ïù¥Îã§. Îî∞ÎùºÏÑú, 1ÎÉâÎèôÌÜ§ÏùÄ 24ÏãúÍ∞Ñ ÎèôÏïà 0‚ÑÉÏùò Î¨º 1ÌÜ§ÏùÑ 0‚ÑÉÏùò ÏñºÏùåÏúºÎ°ú ÎßåÎìúÎäî ÎÉâÎèôÎä•Î†•Ïù¥Îã§.

---


2Ïû•-201Î≤à. Ï†ÄÌï≠ Í≥ÑÏÇ∞ Î¨∏Ï†ú. Ï†ïÎãµ(34Ï¥à), ÌäπÏàòÎ¨∏Ïûê Œ©. Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ O

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: 90[Œ©] Ï†ÄÌï≠ 3Í∞úÎ•º ÏÑúÎ°ú Ï†ëÏÜçÌïòÏó¨ ÏñªÏùÑ Ïàò ÏûàÎäî Í∞ÄÏû• ÏûëÏùÄ Ï†ÄÌï≠Í∞íÏùÄ? Í∞Ä. 90[Œ©] ÎÇò. 15[Œ©] ÏÇ¨. 30[Œ©] ÏïÑ. 60[Œ©]

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: ÏµúÏã† Î¨∏Ï†ú 08Ìöå
Í∞Ä. 1. 0. 0, o ÎÇò. o, 1, o, o 19‚ñ† Îã§ÏùåÍ≥º Í∞ôÏùÄ Îã®Ïàú Ïª§Ìå®ÏãúÌÑ¥Ïä§ Î∏åÎ¶¨ÏßÄÌöåÎ°úÏóêÏÑú
ÏÇ¨. 0, 0, 1, 0 ÏïÑ. 0, 0, 0, 1
16‚ñ†Ï†ÑÏïï Ï°∞Ï†ïÍ∏∞ ÌöåÎ°úÏóêÏÑú Î∂ÄÌïòÏ†ÄÌï≠ rl ÏñëÎã® 
Ïóê Í±∏Î¶¨Îäî Ï†ÑÏïïÏùÄ ÏïΩ Î™á MÏù∏Í∞Ä?
Ïúº ‰∫∫ R
>. ~...
Î¨∏ÏÑú 2: 200 ‚ñ∫ Î¶¥Î¶¨ÌîÑ Î∞∏Î∏åÏùò ÏÑ§Î™Ö Ï§ë ÎßûÎäî Í≤ÉÏùÄ?
Í∞Ä. Ïú†ÏïïÏùÑ ÏùºÏ†ïÏπòÎ°ú Ïú†ÏßÄÌïúÎã§.
ÎÇò. Ïú†ÏïïÏùÑ ÏùºÏ†ïÏπò Ïù¥ÏÉÅÏúºÎ°ú Ïú†ÏßÄÌïúÎã§.
‰∫∫>. Ïú†Ï≤¥Ïùò Î∞©Ìñ•ÏùÑ Ï†úÏñ¥ÌïúÎã§.
ÏïÑ. Ïú†ÎüâÏùÑ Ï°∞Ï†ïÌïúÎã§.
/ Î¶¥Î¶¨ÌîÑ Î∞∏Î∏åÎäî ÏïïÎ†•Ïù¥ ÏÑ§Ï†ï ÏïïÎ†•Ïóê ÎèÑÎã¨ÌïòÎ©¥ 
Ïú†Ï≤¥Î•º Ïö∞ÌöåÏãúÏºú ÏïïÎ†•ÏùÑ ÏùºÏ†ïÏπòÎ°ú Ïú†ÏßÄÌïúÎã§.
20...
Î¨∏ÏÑú 3: Ïù¥ÎèôÌïòÎ©¥ ÏõêÌåêÏù¥ Í∞ôÏùÄ Î∞©Ìñ•ÏúºÎ°ú ÌöåÏ†ÑÌïúÎã§. Ïù¥Í≤É 
ÏùÄ ÏûêÏÑùÏùò ÏïÑÎèôÏóê ÏùòÌï¥ ÏõêÌåêÏÉÅÏóê Î∞úÏÉùÌïòÎäî ÏôÄÏ†ÑÎ•ò 
ÏôÄ ÏûêÏÜçÍ≥ºÏùò ÏÇ¨Ïù¥Ïùò Ï†ÑÏûêÎ†•Ïóê ÏùòÌï¥ ÌÜ†ÌÅ¨Î•º Î∞úÏÉùÌïò 
Îäî Í≤ÉÏúºÎ°ú, Ïú†ÎèÑ Ï†ÑÎèôÍ∏∞Îäî Ïù¥ ÏõêÎ¶¨Î•º Ïù¥Ïö©ÌïòÏó¨ 
ÏûêÏÑùÏùÑ ÏõÄÏßÅÏù¥Îäî ÎåÄÏã† ÌöåÏ†Ñ ÏûêÍ≥ÑÎ•º Ï£ºÏñ¥ÏÑú ÌöåÏ†Ñ 
ÌÜ†ÌÅ¨Î•º ÏñªÎäî Í≤ÉÏù¥Îã§.
0...
Î¨∏ÏÑú 4: ÏµúÏã† Î¨∏Ï†ú 10Ìöå
‚Äò0‚ÄôÏùÑ Í∞ÄÎ¶¨ÌÇ®Îã§Î©¥ R4Ïùò Ï†ÄÌï≠ÏùÄ Î™á [kQ]Ïù∏Í∞Ä?
Í∞Ä. 4[kQ] ÎÇò. 6[kQ]
ÏÇ¨. 8[kQ] ÏïÑ. 10[kQ]
Í∞Ä. 6.2[kW] ÎÇò. 8.8[kW]
ÏÇ¨. 12.2[kW] ÏïÑ. 16.2[kW]
07‚Ä¢Î≥ÄÏïïÍ∏∞ÏóêÏÑú Í∂åÏÑ†ÎπÑÏôÄ Í∞ôÏùÄ Í∞íÏùÑ Í∞ñÎäî Í≤É
ÏùÄ?
...
Î¨∏ÏÑú 5: ÏµúÏã† Î¨∏Ï†ú
ÎÇòÎàà Í∞í
ÎÇò. ÏÑ†Ï≤¥Ïùò Ï†ÑÏßÑÏÜçÎèÑÎ•º Î∞òÎ•òÏùò ÏÜçÎèÑÎ°ú ÎÇòÎàà Í∞í 
ÏÇ¨. Î∞òÎ•òÏùò ÏÜçÎ†•ÏùÑ Ï∂îÏßÑÍ∏∞Ïùò ÏÜçÎ†•ÏúºÎ°ú ÎÇòÎàà Í∞í 
ÏïÑ. Ï∂îÏßÑÍ∏∞Ïùò ÏÜçÎ†•ÏùÑ Î∞òÎ•òÏùò ÏÜçÎ†•ÏúºÎ°ú ÎÇòÎàà Í∞í
23. Î∞∞Ïùò Ï°∞ÌååÏ†ÄÌï≠Ïóê Í∞ÄÏû• ÌÅ∞ ÏòÅÌñ•ÏùÑ Ï£ºÎäî Í≤ÉÏùÄ?
Í∞Ä. EulerÏàò ÎÇò. FroudeÏàò
ÏÇ¨. WeberÏàò...
Î¨∏ÏÑú 6: Ïù¥ ÌïÑÏöîÌïúÍ∞Ä? ÔºàÎã®, Ìïú ÏÇ¨ÎûåÏùÄ ÌèâÍ∑† 3Ïù¥ kgfl Ïùò ÌûòÏúº 
Î°ú Î∞ÄÍ±∞ÎÇò ÎÅå Ïàò ÏûàÎã§Í≥† ÌïúÎã§.Ôºâ
Í∞Ä. 3Î™Ö ÎÇò. 6Î™Ö
‰∫∫E 9Î™Ö ÏïÑ. 27Î™Ö
/ ÎßàÏ∞∞Í≥ÑÏàòÍ∞Ä 0.2 Ïù¥ÎØÄÎ°ú 800 x 0.2 / 30 = 5.48 
Ïù¥ÎØÄÎ°ú ÏµúÏÜå 6Î™ÖÏù¥Îã§.
045 ‚ñ∫ 427[PS]Ïùò ÏõêÎèôÍ∏∞Í∞Ä...
Î¨∏ÏÑú 7: Ï†ú 9 Ìöå [ Í∏∞Í¥Ä m ]
01. Îã§ÏùåÍ≥º Í∞ôÏùÄ ÌúòÌä∏Ïä§ÌÜ§ Î∏åÎ¶¨ÏßÄ(Wheatstone 
bridge) ÌöåÎ°úÏóêÏÑú Ï†ÄÌï≠ RxÏùò Í∞íÏùÄ? (Îã®, R, 
=1Ïù¥‚ëµ, R2=2Ïù¥‚ëµ, R3=3Ïù¥Ïù¥Ïù¥Í≥† Í≤ÄÎ•ò 
Í≥Ñ GÏóêÎäî Ï†ÑÎ•òÍ∞Ä ÌùêÎ•¥ÏßÄ ÏïäÎäî ÏÉÅÌÉúÏù¥Îã§.) 
Í∞Ä. 2Ïù¥Ïù¥ ÎÇò. 4Ïù¥Ïù¥
ÏÇ¨. 5(...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer:
**Final Answer :** ÏÇ¨. 30[Œ©]

**Ìï¥ÏÑ§:**
Î¨∏Ï†úÏóêÏÑú 90[Œ©] Ï†ÄÌï≠ 3Í∞úÎ•º ÏÑúÎ°ú Ï†ëÏÜçÌïòÏó¨ ÏñªÏùÑ Ïàò ÏûàÎäî Í∞ÄÏû• ÏûëÏùÄ Ï†ÄÌï≠Í∞íÏùÑ Î¨ªÍ≥† ÏûàÏäµÎãàÎã§. Ï†ÄÌï≠ÏùÑ ÏßÅÎ†¨Î°ú Ïó∞Í≤∞ÌïòÎ©¥ Ï†ÄÌï≠Í∞íÏùÄ Îçî Ïª§ÏßÄÍ≥†, Î≥ëÎ†¨Î°ú Ïó∞Í≤∞ÌïòÎ©¥ Ï†ÄÌï≠Í∞íÏùÄ Îçî ÏûëÏïÑÏßëÎãàÎã§. Îî∞ÎùºÏÑú 3Í∞úÏùò Ï†ÄÌï≠ÏùÑ Î≥ëÎ†¨Î°ú Ïó∞Í≤∞ÌïòÎ©¥ Í∞ÄÏû• ÏûëÏùÄ Ï†ÄÌï≠Í∞íÏùÑ ÏñªÏùÑ Ïàò ÏûàÏäµÎãàÎã§. 3Í∞úÏùò Ï†ÄÌï≠ÏùÑ Î≥ëÎ†¨Î°ú Ïó∞Í≤∞ÌïòÎ©¥, Í∞Å Ï†ÄÌï≠Ïùò Ï†ÄÌï≠Í∞íÏùÄ 90[Œ©] / 3 = 30[Œ©]Ïù¥ Îê©ÎãàÎã§. Îî∞ÎùºÏÑú 3Í∞úÏùò Ï†ÄÌï≠ÏùÑ Î≥ëÎ†¨Î°ú Ïó∞Í≤∞ÌñàÏùÑ Îïå ÏñªÏñ¥ÏßÄÎäî Í∞ÄÏû• ÏûëÏùÄ Ï†ÄÌï≠Í∞íÏùÄ 30[Œ©]ÏûÖÎãàÎã§.

---


3Ïû•-74Î≤à. ÌïúÏòÅ ÌòºÌï© Î¨∏Ï†ú. Ï†ïÎãµ(24Ï¥à), Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: Ï¥àÏùåÌåå Ïú†ÏÜçÍ≥ÑÎäî Ïñ¥Îñ§ Ìö®Í≥ºÎ•º Ïù¥Ïö©Ìïú Í≤ÉÏù∏Í∞Ä? Í∞Ä. Ï†úÎ≤° Ìö®Í≥º (Seebeck effect) ÎÇò. ÏïïÏ†Ñ Ìö®Í≥º ÏÇ¨. ÌôÄ Ìö®Í≥º (Hall effect) ÏïÑ. ÎèÑÌîåÎü¨ Ìö®Í≥º (Doppler effect)

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: ÌéÑÏä§Í∞Ä ÏñªÏñ¥ÏßÑÎã§.
073 ‚ñ∫ Îã§Ïù¥Ïò§Îìú Ïó¨Îü¨ Í∞úÎ•º ÏßÅÎ†¨Î°ú Ïó∞Í≤∞ÌïòÎ©¥?
Í∞Ä. Îçî ÎÜíÏùÄ Ïó≠Î∞©Ìñ•Ï†ÑÏïïÏóê Í≤¨Îîú Ïàò ÏûàÎã§. 
ÎÇò. Îçî ÎßéÏùÄ Î∂ÄÌïòÏ†ÑÎ•òÎ•º ÌùòÎ¶¥ Ïàò ÏûàÎã§. 
ÏÇ¨. Îß•ÎèôÏù¥ Îçî ÏûëÏùÄ Ï†ïÎ•òÌååÌòïÏùÑ ÏñªÏùÑ Ïàò ÏûàÎã§. 
ÏïÑ. ÏàúÎ∞©Ìñ•Ï†ÑÏïï ÌÅ¨Í∏∞Î•º Í∞êÏÜåÏãúÌÇ¨ Ïàò ÏûàÎã§.
/ Îã§Ïù¥Ïò§ÎìúÎäî Ïó≠Î∞©Ìñ• ...
Î¨∏ÏÑú 2: Ï†ÑÎ•òÎ•º ÌùòÎ¶∞Îã§.
102‚ñ∫ ÍµêÎ•òÌöåÎ°úÏóêÏÑú Ïú†Ìö®Ï†ÑÎ†•Ïù¥ 3[w], Ìîº 
ÏÉÅÏ†ÑÎ†•Ïù¥ 5„ÄêVA]Ïùº Îïå Ïó≠Î•†ÏùÄ ÏñºÎßàÏù∏Í∞Ä?
Í∞Ä. 0.5 ÎÇò. 0.6 ‰∫∫?. 0.7 ÏïÑ. 0.8
/ Ïó≠Î•† = Ïú†Ìö®Ï†ÑÎ†• / ÌîºÏÉÅÏ†ÑÎ†•
103 ‚ñ∫ Îã§Ïù¥Ïò§Îìú Ïó¨Îü¨ Í∞úÎ•º ÏßÅÎ†¨Î°ú Ïó∞Í≤∞ÌïòÎ©¥?
Í∞Ä. Îçî ÎÜíÏùÄ Ïó≠Î∞©Ìñ•Ï†ÑÏïïÏóê ...
Î¨∏ÏÑú 3: Í∞Ä. Î≤®Î°úÏä§ ÎÇò. Îã§Ïù¥Ïñ¥ÌîÑÎû® 
ÏÇ¨. ÎÖ∏Ï¶êÍ≥º ÌîåÎûòÌçº ÏïÑ. ÎåÄÏãúÌè¨Ìä∏
/ ÎÖ∏Ï¶êÍ≥º ÌîåÎûòÌçº : ÎØ∏ÏÑ∏Ìïú Î≥ÄÏúÑÎ•º Í≥µÍ∏∞ÏïïÏã†Ìò∏Î°ú 
Î≥ÄÌôòÌïòÎäî Í∏∞Íµ¨
066 ‚ñ∫ Îã§Ïùå Ï§ë Ïó¥Ï†ÑÎåÄÏôÄ Í¥ÄÍ≥ÑÍ∞Ä ÍπäÏùÄ Í≤ÉÏùÄ?
Í∞Ä. Ï†úÎ≤° Ìö®Í≥†KSeebeck effect)Î•º Ïù¥Ïö©ÌïúÎã§. 
ÎÇò. ÏïïÏ†Ñ Ìö®Í≥ºÎ•º Ïù¥Ïö©ÌïúÎã§.
ÏÇ¨....
Î¨∏ÏÑú 4: ÏßÄÎ•º Î∞òÎ≥µÌïòÎäî Îç∞ ÌïÑÏöîÌïú Í≤ÄÏ∂úÍ∏∞Îäî?
Í∞Ä. Î†àÎ≤® Ïä§ÏúÑÏπò ÎÇò. Ïò®ÎèÑ Ïä§ÏúÑÏπò
ÏÇ¨. ÏïïÎ†• Ïä§ÏúÑÏπò ÏïÑ. Î¶¨Î∞ã Ïä§ÏúÑÏπò
/ ÏïïÎ†•Ïä§ÏúÑÏπòÏóêÏÑú ÏïïÎ†•Ïù¥ ÎÇÆÏïÑÏßÄÎ©¥ ÏïïÏ∂ïÍ∏∞Î•º Í∏∞Îèô 
ÏãúÏºú ÏïïÎ†•ÌÉ±ÌÅ¨Ïóê Í≥µÍ∏∞Î•º Ï±ÑÏö∞Í≥† ÏïïÎ†•Ïù¥ ÎÜíÏïÑÏßÄÎ©¥ 
ÏïïÏ∂ïÍ∏∞Í∞Ä Î©àÏ∂îÎèÑÎ°ù ÌïúÎã§.
198 ‚ñ∫ Ïú†ÏïïÏù¥ ÏùºÏ†ïÏïïÎ†• Ïù¥ÏÉÅÏù¥ ÎêòÏóà...
Î¨∏ÏÑú 5: Í≤ΩÏö∞ÏóêÎäî ÏùºÎ∞ò Îã§Ïù¥Ïò§ÎìúÏôÄ Í∞ôÏùÄ ÌäπÏÑ±ÏùÑ Í∞ÄÏßÄÎ©∞, 
Ïó≠Î∞©Ìñ•Ï†ÑÏïïÏùº Í≤ΩÏö∞ÏóêÎäî ÏùºÎ∞òÏ†ÅÏúºÎ°ú Ï†ÑÎ•òÍ∞Ä ÌùêÎ•¥ 
ÎèÑÎ°ù ÌïòÎäî Ïû•Ïπò
142 ‚ñ∫ Í¥ëÏ†ÑÏÑºÏÑúÏùò Ïû•Ï†êÏù¥ ÏïÑÎãå Í≤ÉÏùÄ?
Í∞Ä. ÏûêÍ≥ÑÏùò ÏòÅÌñ•ÏùÑ Î∞õÏßÄ ÏïäÎäîÎã§.
ÎÇò. ÏàòÎ™ÖÏù¥ Í∏∏Îã§.
ÏÇ¨. Ï£ºÏúÑÏùò Í¥ëÎÖ∏Ïù¥Ï¶àÏóê Í∞ïÌïòÎã§.
ÏïÑ. Í≤ÄÏ∂úÍ±∞Î¶¨Í∞Ä ÎπÑÍµêÏ†Å Í∏∏Îã§
/...
Î¨∏ÏÑú 6: 02. ÌèâÏÉÅÏãúÏóê Ïù¥ÏÉÅÏóÜÏù¥ ÏûëÎèôÌïòÎçò Ïú†ÎèÑÏ†ÑÎèôÍ∏∞Í∞Ä Í∏∞ 
ÎèôÎêòÏßÄ ÏïäÎäîÎã§Î©¥ Í∞ÄÏû• Ïö∞ÏÑ†Ï†ÅÏúºÎ°ú Ï†êÍ≤ÄÌï¥Ïïº 
Ìï† Í≤ÉÏùÄ?
Í∞Ä. Ï†ÑÏïï Ï∏°Ï†ï ÎÇò. Ï£ºÌååÏàò Ï∏°Ï†ï 
ÏÇ¨. Ï†àÏó∞Ï†ÄÌï≠ Ï∏°Ï†ï ÏïÑ. Ï†ÑÎ•ò Ï∏°Ï†ï
03.1 Ïù¥ÔºÅÂêåÏùò Ï†ïÏ†ÑÏö©ÎüâÏùÑ Í∞ÄÏßÑ Ï∂ïÏ†ÑÍ∏∞(ÏΩòÎç¥ÏÑú) Îëê 
Í∞úÎ•º ÏßÅÎ†¨ Ï†ëÏÜçÌïòÎ©¥ Ìï©ÏÑ± Ï†ïÏ†ÑÏö©ÎüâÏùÄ ...
Î¨∏ÏÑú 7: Ìùê
Îìú
Í∞Ä. Ïó¥Ï†ÑÎåÄ ÎÇò. ÌîºÌÜ†Í¥Ä
ÏÇ¨. Î∞îÏù¥Î©îÌÉà ÏïÑ. ÏÑúÎØ∏Ïä§ÌÑ∞
17‚ñ†Ïñ¥Îñ§ Î¨ºÎ¶¨ÎüâÏùÑ ÏàòÎüâÏ†ÅÏúºÎ°ú ÌëúÏãúÌïòÎ†§Î©¥ Í∑∏ 
ÏñëÍ≥º Í∞ôÏùÄ Ï¢ÖÎ•òÏùò Í∏∞Ï§ÄÏù¥ ÌïÑÏöîÌïúÎç∞ Ïù¥ ÎπÑÍµêÍ∏∞ 
Ï§ÄÏúºÎ°ú ÏÇ¨Ïö©ÎêòÎäî ÏùºÏ†ïÌïú ÌÅ¨Í∏∞Ïùò ÏñëÏùÑ Î¨¥ÏóáÏù¥ 
Îùº ÌïòÎäîÍ∞Ä?
Í∞Ä. Ïò§Ï∞® ÎÇò. Ï∏°Ï†ï ÏÇ¨. Îã®ÏúÑ ÏïÑ. Î≥¥Ï†ï
18‚ñ† Î∞∞Ï†ÑÎ∞òÏùò...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer Format:
**Final Answer :** ÏïÑ. ÎèÑÌîåÎü¨ Ìö®Í≥º

**Ìï¥ÏÑ§:**
ÏßàÎ¨∏ÏùÄ Ï¥àÏùåÌåå Ïú†ÏÜçÍ≥ÑÍ∞Ä Ïñ¥Îñ§ Ìö®Í≥ºÎ•º Ïù¥Ïö©ÌïòÎäîÏßÄ Î¨ªÍ≥† ÏûàÏäµÎãàÎã§. ContextÏóêÏÑú Ï¥àÏùåÌåå Ïú†ÏÜçÍ≥ÑÏóê ÎåÄÌïú ÏÑ§Î™ÖÏùÄ "ÎèÑÌîåÎü¨ Ìö®Í≥º : ÌååÎèôÏùÑ Î∞úÏÉùÏãúÌÇ§Îäî ÌååÏõêÍ≥º Í∑∏ ÌååÎèôÏùÑ Í¥ÄÏ∏°ÌïòÎäî Í¥ÄÏ∏°Ïûê Ï§ë ÌïòÎÇò Ïù¥ÏÉÅÏù¥ Ïö¥ÎèôÌïòÍ≥† ÏûàÏùÑ Îïå Î∞úÏÉùÌïòÎäî Ìö®Í≥ºÎ°ú, ÌååÏõêÍ≥º Í¥ÄÏ∏°Ïûê ÏÇ¨Ïù¥Ïùò Í±∞Î¶¨Í∞Ä Ï¢ÅÏïÑÏßà ÎïåÏóêÎäî ÌååÎèôÏùò Ï£ºÌååÏàòÍ∞Ä Îçî ÎÜíÍ≤å, Í±∞Î¶¨Í∞Ä Î©ÄÏñ¥Ïßà ÎïåÏóêÎäî ÌååÎèôÏùò Ï£ºÌååÏàòÍ∞Ä Îçî ÎÇÆÍ≤å"ÎùºÍ≥† ÏÑ§Î™ÖÌïòÍ≥† ÏûàÏäµÎãàÎã§. Îî∞ÎùºÏÑú Ï¥àÏùåÌåå Ïú†ÏÜçÍ≥ÑÎäî ÎèÑÌîåÎü¨ Ìö®Í≥ºÎ•º Ïù¥Ïö©Ìï©ÎãàÎã§.

---


4Ïû•-226Î≤à. ÌïúÏòÅ + Ïä∑Ïûê ÌòºÌï©. Ïò§Îãµ(37Ï¥à), Î¶¨Ìä∏Î¶¨Î≤Ñ O, Ìï¥ÏÑ§ X

[6Îã®Í≥Ñ] RAG Ï≤¥Ïù∏ Ïã§Ìñâ
üí¨ ÏßàÎ¨∏: Ìï¥ÏñëÌôòÍ≤ΩÍ¥ÄÎ¶¨Î≤ïÏóê ÏùòÌïú Í∏∞Î¶ÑÏò§ÏóºÎ∞©ÏßÄÏÑ§ÎπÑ Ï§ë Í∏∞Î¶ÑÎ∞õÏù¥Ïùò Í∏∞Ï§ÄÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ? Í∞Ä. Ïû¨ÏßàÏùÄ Í∏àÏÜçÏ†úÏùº Í≤É ÎÇò. ÏÑ†Î∞ïÏù¥ ÏÑ∏Î°úÎ∞©Ìñ•ÏúºÎ°ú 10ÎèÑ ÎòêÎäî Í∞ÄÎ°úÎ∞©Ìñ•ÏúºÎ°ú 22.5ÎèÑ Í≤ΩÏÇ¨ÌïòÎäî Í≤ΩÏö∞ÏóêÎèÑ Í∏∞Î¶ÑÏù¥ ÎÑòÏπòÍ±∞ÎÇò ÌùêÎ•¥ÏßÄ ÏïäÎäî Íµ¨Ï°∞Ïùº Í≤É ÏÇ¨. Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ÏÑ§Ïπò ÏïÑ. Î™®Îì† ÌÉ±ÌÅ¨Ïùò ÏóêÏñ¥Î≤§Ìä∏(tank air vent)ÏóêÎäî Î∞òÎìúÏãú Í∏∞Î¶Ñ Î∞õÏù¥Í∞Ä ÏûàÏùÑ Í≤É

--- [Í≤ÄÏÉâÎêú Î¨∏ÏÑú (Context)] ---
Î¨∏ÏÑú 1: ÎπÑ Ï§ë Í∏∞Î¶ÑÎ∞õÏù¥Ïùò Í∏∞Ï§ÄÏúºÎ°ú Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ? 
Í∞Ä. Ïû¨ÏßàÏùÄ Í∏àÏÜçÏ†úÏùº Í≤É
ÎÇò. ÏÑ†Î∞ïÏù¥ ÏÑ∏Î°úÎ∞©Ìñ•ÏúºÎ°ú 10ÎèÑ ÎòêÎäî Í∞ÄÎ°úÎ∞©Ìñ• 
ÏúºÎ°ú 22.5ÎèÑ Í≤ΩÏÇ¨ÌïòÎäî Í≤ΩÏö∞ÏóêÎèÑ Í∏∞Î¶ÑÏù¥ 
ÎÑòÏπòÍ±∞ÎÇò ÌùêÎ•¥ÏßÄ ÏïäÎäî Íµ¨Ï°∞Ïùº Í≤É
ÏÇ¨. Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ 
ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ...
Î¨∏ÏÑú 2: ÏßÅÎ¨¥ÏùºÎ∞ò Ï≤ôÏ§ë ÏòàÏÉÅ Î¨úÏ°î
Í∞Ä ÏàòÎ©¥ÏúÑÎ°ú Ïò¨ÎùºÏò¨ Í∞ÄÎä•ÏÑ±Ïù¥ ÏûàÏñ¥ ÏÑ†ÎØ∏Î•º Î¨¥Í≤ÅÍ≤å 
ÌïòÍ∏∞ ÏúÑÌï¥ Î∞∞ÏàòÌïòÏó¨ÏÑúÎäî ÏïàÎêúÎã§.
223 ‚ñ∫ Ïó∞Î£åÏú† Ï†ÄÏû•ÌÉ±ÌÅ¨Ïùò Ï†êÍ≤ÄÏ∞® ÌÉ±ÌÅ¨ ÎÇ¥Î∂ÄÏóê 
Îì§Ïñ¥Í∞ÄÎäî Í≤ΩÏö∞Ïùò Ï£ºÏùòÏÇ¨Ìï≠ Ï§ë Ïò≥ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ?
Í∞Ä. ÏÜ°ÌíçÏû•ÏπòÎ°ú ÌÉ±ÌÅ¨ ÎÇ¥Î∂ÄÏùò Í∞ÄÏä§Î•º Ï†úÍ±∞Ìïú 
ÌõÑ Îì§Ïñ¥Í∞à Í≤É
ÎÇò. ...
Î¨∏ÏÑú 3: 217 ‚ñ∫ Îã§Ïùå Ï§ë Í∏∞Î¶ÑÏùò Ìï¥ÏñëÎ∞∞Ï∂úÏù¥ ÌóàÏö©ÎêòÎäî Í≤Ω 
Ïö∞Îäî?
Í∞Ä. Ìï≠Ìï¥ Ï§ë 15[ppm] Ïù¥ÏÉÅÏùò Ïú†Î∂ÑÎÜçÎèÑÏùò 
Ï≤òÎ¶¨ÏàòÎ°ú Î∞∞Ï∂úÌï† Í≤ΩÏö∞
ÎÇò. ÏòÅÌï¥Í∏∞ÏÑ†ÏúºÎ°úÎ∂ÄÌÑ∞ 25Ìï¥Î¶¨ Ïù¥ÏÉÅÏóêÏÑú Ìï≠ 
Ìï¥ Ï§ëÏóê Í∏∞Í¥ÄÏã§ ÎπåÏßÄÌéåÌîÑÎ•º ÌÜµÌï¥ Î∞∞Ï∂úÌïò 
Îäî Í≤ΩÏö∞
ÏÇ¨. Í∏∞Í¥ÄÏã§ Í∏∞Í∏∞Î•º ÏàòÎ¶¨Ìïú ÌõÑ Î∞úÏÉùÌïú ÌèêÏú†Î•º ...
Î¨∏ÏÑú 4: s
ÏïÑ. Í∏∞Î¶Ñ ÌÉ±ÌÅ¨ ÏÜçÏóêÏÑú ÏûëÏóÖÏùÑ Ìï† ÎïåÏóêÎäî ÎØ∏Î¶¨ 
Ï∂©Î∂ÑÌûà ÌôòÍ∏∞ÏãúÏºúÏïº ÌïúÎã§.
/ Î™∏ÏùÄ Ï†ÑÎ•òÍ∞Ä ÌùêÎ•¥Îäî ÎèÑÏ≤¥ Ïù¥ÎØÄÎ°ú Ïò∑ÏùÑ Í∏∏Í≤å ÏûÖÏñ¥ 
Ï†ÑÍ∏∞Í∞Ä ÌÜµÌïòÏßÄ ÏïäÎèÑÎ°ù ÌïúÎã§.
162 ‚ñ∫ Îã§Ïùå Ï§ë Í∏∞Î¶Ñ ÎòêÎäî Ïú†Ìï¥Ïï°Ï≤¥Î¨ºÏßàÏùò Ìï¥Ïñë 
Ïò§ÏóºÎπÑÏÉÅÍ≥ÑÌöçÏÑúÍ∞Ä ÎπÑÏπòÎêòÏñ¥Ïïº Ìï† ÏÑ†Î∞ïÏù¥ ÏïÑ 
Îãå Í≤ÉÏùÄ?
Í∞Ä....
Î¨∏ÏÑú 5: ÎÇò. Í∏∞Î¶ÑÏú†Ï∂úÏÇ¨Í≥† Î∞úÏÉù Ïãú Ìï¥Îãπ ÏÑ†Î∞ïÏùò Í∏∞Í¥ÄÏû• 
Ïù¥ Ï∑®Ìï¥Ïïº Ìï† Î≥¥Í≥† Ï†àÏ∞®Ïóê Í¥ÄÌïú ÏÇ¨Ìï≠
‰∫∫1. ÏÑ†ÏõêÏùò Î∞©Ï†úÍµêÏú° ‚Ä¢ ÌõàÎ†®Ïóê Í¥ÄÌïú ÏÇ¨Ìï≠ 
ÏïÑ. Í∏∞Î¶ÑÏú†Ï∂úÏùÑ Ï§ÑÏù¥Í∏∞ ÏúÑÌïòÏó¨ Ìï¥Îãπ ÏÑ†Î∞ïÏùò 
ÏÑ†ÏõêÏù¥ Ï∑®Ìï† Î∞©Ï†úÏ°∞ÏπòÏóê Í¥ÄÌïú ÏÇ¨Ìï≠
/ ÏÑ†Î∞ïÏóêÏÑúÏùò Ïò§ÏóºÎ∞©ÏßÄÏóê Í¥ÄÌïú Í∑úÏπô Ï†ú25Ï°∞ÔºàÏÑ†Î∞ï 
Ìï¥ÏñëÏò§...
Î¨∏ÏÑú 6: Î∂ÄÌÑ∞ Ï†ú47Ï°∞ÍπåÏßÄ Î∞è Ï†ú47Ï°∞Ïùò2Îäî Îã§Ïùå Í∞Å Ìò∏Ïùò 
Ïñ¥Îäê ÌïòÎÇòÏóê Ìï¥ÎãπÌïòÎäî Í≤ΩÏö∞ÏóêÎäî Ï†ÅÏö©ÌïòÏßÄ ÏïÑÎãàÌïúÎã§.
1. ÏÑ†Î∞ï Î∞è Ìï¥ÏñëÏÇ¨ÏÑ§Ïùò ÏïàÏ†Ñ ÌôïÎ≥¥ ÎòêÎäî Ïù∏Î™ÖÍµ¨Ï°∞Î•º 
ÏúÑÌïòÏó¨ Î∂ÄÎìùÏù¥ÌïòÍ≤å ÎåÄÍ∏∞Ïò§ÏóºÎ¨ºÏßàÏù¥ Î∞∞Ï∂úÎêòÎäî 
Í≤ΩÏö∞
2. ÏÑ†Î∞ï ÎòêÎäî Ìï¥ÏñëÏãúÏÑ§Ïùò ÏÜêÏÉÅ Îì±ÏúºÎ°ú Ïù∏ÌïòÏó¨ 
Î∂ÄÎìùÏù¥ÌïòÍ≤å ÎåÄÍ∏∞Ïò§...
Î¨∏ÏÑú 7: Ìûà Î∞∞Ï∂úÌïúÎã§.
ÏÇ¨. ÏûêÎèôÏ†úÏñ¥Í∏∞Í∏∞ÏôÄ ÏõêÍ≤©Ï†úÏñ¥Í∏∞Í∏∞Ïùò ÏûëÎèôÏÉÅ 
ÌÉúÎ•º Ïû¨ÌôïÏù∏ÌïúÎã§.
ÏïÑ. Ï∂îÏßÑÍ∏∞Í¥ÄÏùò Í≥µÌöåÏ†ÑÏùÑ ÌîºÌïòÍ∏∞ ÏúÑÌï¥ ÏÑ†ÎØ∏ÌÉ± 
ÌÅ¨Î•º Î∞∞ÏàòÌïúÎã§.
04. ÌîºÎ≥µÏïÑÌÅ¨ Ïö©Ï†ëÏóê ÎåÄÌïú Ïû¨Ìï¥Î∞©ÏßÄ ÎåÄÏ±ÖÏúºÎ°ú Ïò≥ 
ÏßÄ ÏïäÏùÄ Í≤ÉÏùÄ?
Í∞Ä. ÎààÏùÑ Î≥¥Ìò∏ÌïòÍ∏∞ ÏúÑÌï¥ Ï∞®Í¥ëÎ≥¥Ìò∏Î©¥ÏùÑ ÏÇ¨Ïö© 
ÌïúÎã§.
ÎÇò. Ïö©Ï†ëÍ∏∞...
------------------------------
--- [ÏµúÏ¢Ö ÎãµÎ≥Ä] ---
## Final Answer:
**Final Answer:** ÏÇ¨. Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ÏÑ§Ïπò
**Ìï¥ÏÑ§:** Ìï¥ÏñëÌôòÍ≤ΩÍ¥ÄÎ¶¨Î≤ïÏóê ÏùòÌïú Í∏∞Î¶ÑÎ∞õÏù¥Ïùò Í∏∞Ï§ÄÏùÄ Îã§ÏùåÍ≥º Í∞ôÏäµÎãàÎã§.
1) Ïû¨Ïßà : Í∏àÏÜçÏ†úÏùº Í≤É
2) Íµ¨Ï°∞ : ÏÑ†Î∞ïÏù¥ ÏÑ∏Î°úÎ∞©Ìñ•ÏúºÎ°ú 10ÎèÑ ÎòêÎäî Í∞ÄÎ°úÎ∞©Ìñ•ÏúºÎ°ú 22.5ÎèÑ Í≤ΩÏÇ¨ÌïòÎäî Í≤ΩÏö∞ÏóêÎèÑ Í∏∞Î¶ÑÏù¥ ÎÑòÏπòÍ±∞ÎÇò ÌùêÎ•¥ÏßÄ ÏïäÎäî Íµ¨Ï°∞Ïùº Í≤É
3) ÏÑ§ÏπòÏû•ÏÜå : Ïú†ÌÉ±ÌÅ¨,Ïú†Î©¥Í≥Ñ,Ïú†ÌéåÌîÑ,Ïó¨Í≥ºÍ∏∞,Î≥¥ÏùºÎü¨ÌôîÍµ¨,Ïú†Ï≤≠Ï†ïÍ∏∞,Î∂ÄÏÇ∞Î∞∏Î∏åÏãúÌóòÏû•Ïπò, Ï£ºÍ∏∞Í¥Ä, Î≥¥Ï°∞Í∏∞Í¥Ä, Í∏∞Î¶ÑÏù¥ ÌùêÎ•¥Îäî Í¥ÄÏû•ÏπòÏùò Î∞∏Î∏å ÎòêÎäî ÏΩï Í∑∏ Î∞ñÏóê Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Ïû•ÏπòÏùò Î∞ëÏóê ÏÑ§ÏπòÌï† Í≤É
Îî∞ÎùºÏÑú, Í∏∞Î¶ÑÏù¥ ÏÉàÏñ¥ÎÇòÏò¨ Ïö∞Î†§Í∞Ä ÏûàÎäî Í∞ÅÏ¢Ö Í∏∞Í∏∞ÎÇò ÌÉ±ÌÅ¨(tank)Ïùò Î∞ëÏóê ÏÑ§ÏπòÌïòÎäî Í≤ÉÏùÄ Í∏∞Î¶ÑÎ∞õÏù¥ Í∏∞Ï§ÄÏóê Ìï¥ÎãπÌïòÏßÄ ÏïäÏäµÎãàÎã§.

---



*ÎûúÎç§ Î¨∏Ï†ú Ï†ïÎãµÎ•†
5/8 = 0.625

*bge-m3 ÏûÑÎ≤†Îî©Îî©
Î¶¨Ìä∏Î¶¨Î≤Ñ O

*Gemma-3-n
ÌïúÍµ≠Ïñ¥ Ï∂îÎ°† Î∞è Ìï¥ÏÑ§ X, Ïà´Ïûê Í≥ÑÏÇ∞ O-> Î™®Îç∏ ÏàòÏ†ï ÌïÑÏöî